# School website scraper in four parts: 
- Part 1: Set-Up
    - set up python environment and jupyter notebook 
- Part 2-3: Extracting
    - read csv input file with school addresses
    - visit each school homepage and extract external links,
        - miner config for second loop on all internal links
    - storing data in intermediate csv files
- Part 4-5: Transforming 
    - link extraction
    - link standardization
    - categorization
- Part 6: Loading of Visualizations
    - adding nodes and relations
    - modifying nodes and relations

### Functional requirements:
- parsing an input file
- mining school websites for links
    - visit 1-3 layers of internal links, look for external links
    - store all links in a csv file 
- visualize it as a graph with colors, by using [pyvis](https://pyvis.readthedocs.io/en/latest/documentation.html?highlight=template#pyvis.network.Network.set_template).
- export it as a .html file for webhosting / analysis
- using intermediate files, based on date of access and key factors

### non-functional requirements:
- Easy to set up and follow documentation
- Allows for customizability

### Status
This notebook would need another round of refinement, and some recommended changes are given in the sections. 
Some duplicate or unused code is still available in the categorization section because this was the most difficult challenge.
The quality of this notebook is a proof of concept and does not fulfil my standards of good code. The code can be run and improved in case it is necessary. 

## Part 1: Set-up
- from terminal, set up new environment: 
```python3 -m venv schoolscraperenv ``` 
- activate environment: 
```source schoolscraperenv/bin/activate ``` 
- install kernel that can be used by jupyter: 
```pip install ipykernel ``` 
- activate the created environment as a kernel: 
```python -m ipykernel install --user --name=schoolscraperenv --display-name "Python (schoolscraperenv)" ``` 
- now it should be listed as an active kernel: 
```jupyter kernelspec list ``` 
- install required packages: 
```python3 -m pip install -r requirements.txt ``` 
- launch jupyter: 
```jupyter notebook ``` 
- now when you open the notebook you should be able to select the new kernel

The next cell is complecting the set-up by importing and making available the libraries for the rest of the notebook. 



In [ ]:
# Import libraries
import time 
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urlsplit, urlunsplit
import csv
import pandas

## Part 2: Parse list of schools
Takes a csv with schools as input and cleans up the variables
### Part 2.1: Load schools csv file. 

In [ ]:
## Part 2.1: Load schools csv file. 
## Creates df
# Tries to make a list of all external links from homepages in a csv
import csv
import pandas as pd
from mining_services import get_netloc

csv_path = './public-schooldata/vienna-schools.csv'
csv_path_austria = './public-schooldata/austria-schools.csv'


# Using Panda: Loading the .csv file into a DataFrame
df = pd.read_csv(csv_path)
df_austria = pd.read_csv(csv_path_austria)
# Inspect the DataFrame
print("DataFrame Head:")
# print(df.head())

# Select relevant columns
df = df.iloc[:, [2, 5, 6, 9, 3, 7]]
df.columns = ['name', 'school_type_code', 'school_type_txt', 'website_url', 'address', 'SKZ']

print(df.head())

# Handle missing values
print("\nMissing Values:")
print(df.isna().sum())

df['school_type_code'] = df['school_type_code'].fillna('Unknown')
df['school_type_txt'] = df['school_type_txt'].fillna('Unknown')
df['website_url'] = df['website_url'].fillna('No URL')

print("\nMissing Values After:")
print(df.isna().sum())

# make sure all URL's have a protocol:
def normalize_school_url(url):
    if url == 'No URL':
        return url 
    elif url.startswith(('http://', 'https://')):
        return url  # URL already starts with http:// or https://
    else:
        print(f"added http:// to {url}")
        return 'http://' + url  # prepend http:// to the URL
df['website_url'] = df['website_url'].apply(normalize_school_url)

### Part 2.3: Descriptive Statistics
Regarding school types, Austria vs. Vienna, etc. 

In [ ]:
# Part 2.3: Descriptive Statistics
## Schooltypes Austria vs. Vienna
### Schooltypes counts
print("Austria Categories:")
print(df_austria['TYP'].value_counts())
print("\n\nVienna Categories:")
print(df['school_type_txt'].value_counts())

### Schooltypes counts, aggregated


#### mapping table
aggregate_mapping_vienna = {
    'Volksschule (öffentlich)': 'Volksschule',
    'Volksschule (privat)': 'Volksschule',
    'Mittelschule (öffentlich)': 'Mittelschule',
    'Mittelschule (privat)': 'Mittelschule',
    'Statutschule (privat)': 'Statutschule',
    'Allgemeinbildende höhere Schule (öffentlich)': 'Allgemeinbildende Höhere Schule',
    'Allgemeinbildende höhere Schule (privat)': 'Allgemeinbildende Höhere Schule',
    'Sonderschule (öffentlich)': 'Sonderschule',
    'Sonderschule (privat)': 'Sonderschule',
    'Berufsschule (öffentlich)': 'Berufsschule',
    'Berufsschule (privat)': 'Berufsschule',
    'Humanberufliche Schule (öffentlich)': 'Berufsschule',
    'Humanberufliche Schule (privat)': 'Berufsschule',
    'Kaufmännische Schule (öffentlich)': 'Berufsbildende Mittlere und Höhere Schule',
    'Kaufmännische Schule (privat)': 'Berufsbildende Mittlere und Höhere Schule',
    'LehrerInnenbildende mittlere und höhere Schulen (öffentlich)': 'LehrerInnen- und ErzieherInnenbildung',
    'LehrerInnenbildende mittlere und höhere Schulen (privat)': 'LehrerInnen- und ErzieherInnenbildung',
    'Polytechnische Schule (öffentlich)': 'Polytechnische Schule',
    'Polytechnische Schule (privat)': 'Polytechnische Schule',
    'Technische Schule (öffentlich)': 'Berufsbildende Mittlere und Höhere Schule',
    'Technische Schule (privat)': 'Berufsbildende Mittlere und Höhere Schule',
    'Zentrallehranstalt (öffentlich)': 'Berufsbildende Mittlere und Höhere Schule',
    'Statutschule (öffentlich)': 'Statutschule',
    'Sonstige (öffentlich)': 'Other',
    'Sonstige (privat)': 'Other',
    'Bundesinstitut (öffentlich)': 'Other'
}
aggregate_mapping_austria = {
    'VS': 'Volksschule',
    'NMSH': 'Mittelschule',
    'AHS': 'Allgemeinbildende Höhere Schule',
    'SS': 'Sonderschule',
    'PS': 'Polytechnische Schule',
    'BMHST': 'Berufsbildende Mittlere und Höhere Schule',
    'ASTAT': 'Statutschule',
    'BS': 'Berufsschule',
    'GKS': 'Berufsschule',
    'BMHSK': 'Berufsbildende Mittlere und Höhere Schule',
    'BMHSW': 'Berufsbildende Mittlere und Höhere Schule',
    'LFMS': 'Mittelschule',
    'BSTAT': 'Statutschule',
    'BMHSP': 'LehrerInnen- und ErzieherInnenbildung',
    'LFHS': 'Berufsbildende Mittlere und Höhere Schule',
    'NMSA': 'Allgemeinbildende Höhere Schule',
    'LMS': 'Other',
    'BMHSS': 'Berufsschule'
}
#### some preliminary checks



# Perform the mapping
#print(aggregate_mapping_austria)
df['school_type_mapping'] = df['school_type_txt'].map(aggregate_mapping_vienna)
df_austria['school_type_mapping'] = df_austria['TYP'].map(aggregate_mapping_austria)
print(df['school_type_mapping'].value_counts())
print(df_austria['school_type_mapping'].value_counts())

# check if there are discrepancies
# Merge the two DataFrames on the 'SKZ' column
df_austria['SKZ'] = df_austria['SKZ'].astype(str)
df['SKZ'] = df['SKZ'].astype(str)
merged_df = pd.merge(df_austria, df, on='SKZ', suffixes=('_dfAT', '_dfVIE'))

# Check for discrepancies in the 'school_type_mapping' column
discrepancies = merged_df[merged_df['school_type_mapping_dfAT'] != merged_df['school_type_mapping_dfVIE']]

print(discrepancies.head)
# Print the discrepancies
print(f"\n\n{discrepancies.shape[0]} Discrepancies found:")
print(discrepancies[['SKZ', 'name', 'school_type_mapping_dfAT', 'school_type_mapping_dfVIE']])






In [ ]:
import pandas as pd

csv_path_austria = './public-schooldata/austria-schools.csv'
df_austria = pd.read_csv(csv_path_austria)

# Vienna schools have SKZ starting with '9'
df_vienna = df_austria[df_austria['SKZ'].astype(str).str.startswith('9')]

# Schools from the rest of Austria (SKZ starting with 1-8)
df_austria_rest = df_austria[~df_austria['SKZ'].astype(str).str.startswith('9')]

# Define a function to calculate these statistics
def calculate_school_stats(df, total_schools):
    grouped = df.groupby('TYP')
    
    stats = grouped.size().reset_index(name='Count')
    stats['% Private'] = grouped.apply(lambda x: (x['ERHALTER'] == 'privat').sum() / len(x) * 100).values
    stats['% of Total'] = stats['Count'] / total_schools * 100
    
    return stats

# Total schools count
total_vienna_schools = len(df_vienna)
total_austria_schools = len(df_austria_rest)

# Calculate stats for Vienna and Austria
vienna_stats = calculate_school_stats(df_vienna, total_vienna_schools)
austria_stats = calculate_school_stats(df_austria_rest, total_austria_schools)

# Merge the Vienna and Austria stats into a single DataFrame
merged_stats = vienna_stats.merge(austria_stats, on='TYP', suffixes=('_Vienna', '_Austria'))

# Sort the data by 'TYP' (Category) if needed
merged_stats = merged_stats.sort_values('TYP')

# Calculate total number of private schools and the percentage of private schools
total_vienna_private = (vienna_stats['Count'] * vienna_stats['% Private'] / 100).sum()
vienna_private_percentage = (total_vienna_private / vienna_totals['Count']) * 100

total_austria_private = (austria_stats['Count'] * austria_stats['% Private'] / 100).sum()
austria_private_percentage = (total_austria_private / austria_totals['Count']) * 100

# Update the LaTeX table with the correct totals
latex_table = r"""
\begin{table}[h!]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{l *{2}{rrr}}
    \toprule
    \multirow{2}{*}{\makecell[l]{Category}} & \multicolumn{3}{c}{Vienna} & \multicolumn{3}{c}{Rest of Austria} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    & Schools & \% Private & \% of Schools & Schools & \% Private & \% of Schools \\
    \midrule
"""

# Add rows for each school type
for _, row in merged_stats.iterrows():
    latex_table += f"    {row['TYP']} & {int(row['Count_Vienna'])} & {row['% Private_Vienna']:.1f} & {row['% of Total_Vienna']:.1f} & {int(row['Count_Austria'])} & {row['% Private_Austria']:.1f} & {row['% of Total_Austria']:.1f} \\\\\n"

# Add a thin line before the total row
latex_table += r"    \midrule\n"

# Add the total row with correct private percentage
latex_table += f"    Total & {int(vienna_totals['Count'])} & {vienna_private_percentage:.1f} & {vienna_totals['% of Total']:.1f} & {int(austria_totals['Count'])} & {austria_private_percentage:.1f} & {austria_totals['% of Total']:.1f} \\\\\n"

latex_table += r"""
    \bottomrule
\end{tabular}}
\caption{Distribution of Schools in Vienna and Austria in 2022/2023, based on "Schools in Austria" open dataset}
\label{tab:schools}
\end{table}
"""

# Print or save the LaTeX table code
print(latex_table)



In [ ]:
### Private vs. Public 
# Add new column 'school_status' based on the ending of 'school_type_txt'
df['ERHALTER'] = df['school_type_txt'].apply(lambda x: 'privat' if x.endswith('(privat)') else 'öffentl.')
print(df['ERHALTER'].value_counts())
print(df_austria['ERHALTER'].value_counts())

### Comparison to rest of Austria
vienna_counts = df['school_type_mapping'].value_counts().rename('Vienna_count')
austria_counts = df_austria['school_type_mapping'].value_counts().rename('Austria_count')

# Calculate the relative percentages
vienna_percentages = (vienna_counts / vienna_counts.sum() * 100).rename('Vienna_percentage')
austria_percentages = (austria_counts / austria_counts.sum() * 100).rename('Austria_percentage')

# Combine the counts and percentages into a single DataFrame
comparison_df = pd.concat([vienna_counts, vienna_percentages, austria_counts, austria_percentages], axis=1).fillna(0)

# Print the schooltype comparative statistics
print(comparison_df)

# Now the occurrences of each school type by public and private categories
vienna_counts_erhalter = df.groupby(['ERHALTER', 'school_type_mapping']).size().unstack(fill_value=0).stack().rename('Vienna_count')
austria_counts_erhalter = df_austria.groupby(['ERHALTER', 'school_type_mapping']).size().unstack(fill_value=0).stack().rename('Austria_count')

# Calculate the relative percentages
vienna_totals_erhalter = df['ERHALTER'].value_counts()
austria_totals_erhalter = df_austria['ERHALTER'].value_counts()

vienna_percentages_erhalter = (vienna_counts_erhalter / vienna_totals_erhalter[vienna_counts_erhalter.index.get_level_values('ERHALTER')].values * 100).rename('Vienna_percentage')
austria_percentages_erhalter = (austria_counts_erhalter / austria_totals_erhalter[austria_counts_erhalter.index.get_level_values('ERHALTER')].values * 100).rename('Austria_percentage')

# Combine the counts and percentages into a single DataFrame
comparison_df_erhalter = pd.concat([vienna_counts_erhalter, vienna_percentages_erhalter, austria_counts_erhalter, austria_percentages_erhalter], axis=1).fillna(0)

# Reset index to have a proper DataFrame structure
comparison_df_erhalter.reset_index(inplace=True)

# Print the comparative statistics
print(comparison_df_erhalter)


In [ ]:
## A number of schools in Vienna are extensions of other schools. These have the same name, a different address, and the same school ID (SKZ) as the original school. 
df_extensions = df[df['SKZ'].str.endswith('e')]

# Calculate the number of extensions for each base school
extensions_count = df_extensions.groupby(df['SKZ'].str.rstrip('e')).size()
for skz in extensions_count.index:
    if skz in df['SKZ'].values:
        extensions_count[skz] += 1

# Sort the extensions_count by SKZ
extensions_count = extensions_count.sort_values(ascending=False)

total_sum_extensions = extensions_count.sum()

# Calculate descriptive statistics
average_extensions = extensions_count.mean()
std_dev_extensions = extensions_count.std()

# Print results
print("\nTotal Sum of School Numbers in Extensions Count:")
print(total_sum_extensions)
print("Extensions Count:")
print(extensions_count)
print("\nDescriptive Statistics:")
print(f"Average number of extensions: {average_extensions}")
print(f"Standard deviation of extensions: {std_dev_extensions}")

In [ ]:
print("\nSchool Type Code Frequency:")
print(df['school_type_code'].value_counts())
print(df['school_type_txt'].value_counts())

### Part 2.4: Website URL updating and analysis

In [ ]:
import pandas as pd
import aiohttp
import asyncio
from datetime import datetime
from aiohttp import ClientSession
from urllib.parse import urlparse, urlunparse
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter notebooks
nest_asyncio.apply()

def normalize_url(url):
    if url == "No URL":
        return url
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        parsed_url = parsed_url._replace(scheme="http")
    if parsed_url.scheme not in ["http", "https"]:
        return "Invalid URL"
    return urlunparse(parsed_url)

async def fetch_url(session, url):
    try:
        async with session.get(url, allow_redirects=True) as response:
            if response.status == 200:
                return str(response.url)
            else:
                return "Invalid URL"
    except Exception as e:
        return "Invalid URL"

async def update_urls(df):
    async with ClientSession() as session:
        tasks = []
        for index, row in df.iterrows():
            url = normalize_url(row['website_url'])
            if url not in ["No URL", "Invalid URL"]:
                tasks.append(fetch_url(session, url))
            else:
                tasks.append(asyncio.sleep(0))  # Dummy coroutine for 'No URL' and 'Invalid URL'
        
        results = await asyncio.gather(*tasks)
        # Ensure "No URL" is handled correctly in the updated DataFrame
        for i, result in enumerate(results):
            if df.at[i, 'website_url'] == "No URL":
                results[i] = "No URL"
        df['updated_URL'] = results
    return df

# Function to get netloc from URL and remove www prefix
def get_netloc(url):
    if url in ["No URL", "Invalid URL"]:
        return url
    parsed_url = urlparse(url)
    netloc = parsed_url.netloc
    if netloc and netloc.startswith("www."):
        netloc = netloc[4:]
    return netloc

# Calculate statistics
def get_statistics(df, updated_df):
    total_urls = len(df)
    no_url_count = (df['website_url'] == 'No URL').sum()
    invalid_url_count = (updated_df['updated_URL'] == 'Invalid URL').sum()
    valid_url_count = total_urls - no_url_count - invalid_url_count
    
    redirected_count = (df['website_url'].apply(get_netloc) != updated_df['updated_URL'].apply(get_netloc)).sum()
    
    stats = {
        'total_urls': total_urls,
        'no_url_count': no_url_count,
        'invalid_url_count': invalid_url_count,
        'valid_url_count': valid_url_count,
        'redirected_count': redirected_count
    }
    
    return stats

# Load your actual DataFrame 'df' here
# df = pd.read_csv('your_dataframe.csv')  # Example of loading your actual DataFrame

# Run the async update function
loop = asyncio.get_event_loop()
updated_df = loop.run_until_complete(update_urls(df))

# Display the updated DataFrame
print(updated_df)

# Get statistics
stats = get_statistics(df, updated_df)

# Display the statistics
print("Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

# Compare original and updated DataFrames
comparison_df = df.copy()
comparison_df['updated_URL'] = updated_df['updated_URL']
comparison_df['netloc_changed'] = comparison_df.apply(
    lambda row: get_netloc(row['website_url']) != get_netloc(row['updated_URL']), axis=1
)

print("\nComparison DataFrame:")
print(comparison_df)

# Save the comparison DataFrame to CSV to check out all rows
comparison_df.to_csv(f"2_comparison_df_{datetime.now().strftime('%Y-%m-%d')}.csv", index=False)


In [ ]:
# Loading old data

# Specify the path to the CSV file
csv_path = './2_comparison_df_2024-08-12.csv'

# Read the CSV file into a dataframe
comparison_df = pd.read_csv(csv_path)

In [ ]:
# Descriptive statistics of link availability
netloc_changed_counts = comparison_df["netloc_changed"].value_counts()
updated_URL_invalid_count = comparison_df["updated_URL"].apply(lambda x: x == "Invalid URL").sum()
total_urls = comparison_df.shape[0]

# Percentage of netloc_changed by school type
school_type_group = comparison_df.groupby("school_type_txt")["netloc_changed"].mean() * 100

# Output descriptive statistics
print("Netloc Changed Counts:\n", netloc_changed_counts)
print("Number of Invalid URLs in updated_URL:", updated_URL_invalid_count)
print("Total Number of URLs:", total_urls)
print("Percentage of netloc_changed by school type:\n", school_type_group)

# Grouping data by school type
grouped = comparison_df.groupby("school_type_txt").agg(
    netloc_changed_percentage=("netloc_changed", "mean"),
    observations=("netloc_changed", "count")
)

grouped["netloc_changed_percentage"] = (grouped["netloc_changed_percentage"] * 100).round(2)

# Calculate private and non-private school statistics
private_schools = comparison_df[comparison_df["ERHALTER"] == "privat"]
non_private_schools = comparison_df[comparison_df["ERHALTER"] != "privat"]

private_netloc_changed_count = private_schools["netloc_changed"].sum()
private_total_count = private_schools.shape[0]
private_netloc_changed_percentage = (private_netloc_changed_count / private_total_count * 100).round(2)

non_private_netloc_changed_count = non_private_schools["netloc_changed"].sum()
non_private_total_count = non_private_schools.shape[0]
non_private_netloc_changed_percentage = (non_private_netloc_changed_count / non_private_total_count * 100).round(2)

# Prepare the LaTeX table
latex_table = grouped.to_latex(index=True, 
                               header=["% Netloc Changed", "Observations"], 
                               caption="Netloc Change Statistics by School Type (2021-2024)", 
                               label="tab:netloc_change_stats",
                               column_format="lcc", # Column alignment: left, center, center
                               formatters={"netloc_changed_percentage": "{:.2f}\%".format},
                               escape=False)

# Append private and non-private school statistics
latex_table += "\\hline\n"
latex_table += f"Private Schools & {private_netloc_changed_percentage:.2f}\\% & {private_total_count} \\\\\n"
latex_table += f"Non-Private Schools & {non_private_netloc_changed_percentage:.2f}\\% & {non_private_total_count} \\\\\n"

# Output the LaTeX table
print(latex_table)

In [ ]:
import pandas as pd
from urllib.parse import urlparse

# Assuming `comparison_df` is already loaded and processed as before

# Calculate the total number of schools
total_schools = len(comparison_df)

# Calculate the number of schools with no URL
no_url = comparison_df["website_url"].apply(lambda x: x == "No URL").sum()

# Calculate the percentage of schools with no URL
percentage_no_url = (no_url / total_schools) * 100

# Clean up the domains
def clean_domain(url):
    if url == "No URL":
        return None
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    if domain.startswith("www."):
        domain = domain[4:]
    return domain.rstrip('/')

# Apply the domain cleaning function
comparison_df['clean_domain'] = comparison_df['website_url'].apply(clean_domain)
comparison_df['cleaned_updated_URL'] = comparison_df['updated_URL'].apply(clean_domain)

# Drop rows where the clean_domain is None (for No URL)
valid_domains = comparison_df.dropna(subset=['clean_domain'])

# Count the number of unique domains
unique_domains_count = valid_domains['clean_domain'].nunique()

# Create a dataframe for unique websites
unique_domains_df = valid_domains.drop_duplicates(subset=['clean_domain'])

# Calculate the number of domain changes for all websites
domain_changes_all = valid_domains['netloc_changed'].sum()

# Calculate the number of domain changes for unique websites
domain_changes_unique = unique_domains_df['netloc_changed'].sum()

# Calculate the number of unique domains that became unavailable
unavailable_domains_count = unique_domains_df[unique_domains_df["updated_URL"] == "Invalid URL"].shape[0]

# Calculate the number of schule.wien.at domains that became unavailable
schule_unavailable_count = unique_domains_df[(unique_domains_df["updated_URL"] == "Invalid URL") & 
                                             (unique_domains_df["clean_domain"].str.endswith("schule.wien.at"))].shape[0]

# Calculate the number of unique domains migrating
# Note: This row only has the total column filled with the unique domains number
unique_domains_migrated = domain_changes_unique - unavailable_domains_count

# Calculate the number of domains migrated to schule.wien.at
migrated_to_schule_count = unique_domains_df[(unique_domains_df["netloc_changed"]) & unique_domains_df["cleaned_updated_URL"].str.endswith("schule.wien.at")].shape[0]

# Calculate the number of domains migrated to another unique domain (not schule.wien.at)
migrated_to_other_domain_count = unique_domains_migrated - migrated_to_schule_count

# Calculate the number of domains that migrated from and to schule.wien.at with redirection configured
migrated_within_schule_count = unique_domains_df[(unique_domains_df["clean_domain"].str.endswith("schule.wien.at")) &
                                                 (unique_domains_df["netloc_changed"]) & (unique_domains_df["cleaned_updated_URL"].str.endswith("schule.wien.at"))].shape[0]

# Prepare the LaTeX snippet
latex_snippet = f"""
\\begin{{table}}[ht]
\\centering
\\caption{{Netloc Changes from 15.4.2021 to 13.8.2024}}
\\label{{tab:netloc_changes_domains}}
\\begin{{tabular}}{{lrrr}}
\\toprule
URL Availability Measure & Observations & Total & Percentage \\\\
\\midrule
Total number of schools in dataset & {total_schools} & {total_schools} & 100\\% \\\\
Schools with a website & {total_schools - no_url} & {total_schools} & {100 - percentage_no_url:.2f}\\% \\\\
\\quad Domain changes of all websites & {domain_changes_all} & {total_schools - no_url} & {domain_changes_all / (total_schools - no_url) * 100:.2f}\\% \\\\
Unique domains & {unique_domains_count} & {total_schools} & {unique_domains_count / total_schools * 100:.2f}\\% \\\\
\\quad Domain changes of unique websites & {domain_changes_unique} & {unique_domains_count} & {domain_changes_unique / unique_domains_count * 100:.2f}\\% \\\\
\\quad Unique domains becoming unavailable & {unavailable_domains_count} & {domain_changes_unique} & {unavailable_domains_count / domain_changes_unique * 100:.2f}\\% \\\\
\\quad schule.wien.at domains becoming unavailable & {schule_unavailable_count} & {unavailable_domains_count} & {schule_unavailable_count / unavailable_domains_count * 100:.2f}\\% \\\\
\\midrule
Unique domains migrating &  & {unique_domains_migrated} & \\\\
\\quad Migrated to schule.wien.at domain & {migrated_to_schule_count} & {unique_domains_migrated} & {migrated_to_schule_count / unique_domains_migrated * 100:.2f}\\% \\\\
\\quad Migrated to another unique domain & {migrated_to_other_domain_count} & {unique_domains_migrated} & {migrated_to_other_domain_count / unique_domains_migrated * 100:.2f}\\% \\\\
\\quad Migrated from and to a schule.wien.at domain & {migrated_within_schule_count} & {unique_domains_migrated} & {migrated_within_schule_count / unique_domains_migrated * 100:.2f}\\% \\\\
\\bottomrule
\\end{{tabular}}
\\end{{table}}
"""

print(latex_snippet)


## Part 3: Extracting data from websites
There are four possible approaches:
- Get data from one website homepage
- Get data from whole website domain
- Get data from a series of websites from a .csv file. 
- Get historic data from a series of websites, based on a certain cut-off date. 

The output is stored in a .csv format so it can be analysed in the next stage, where the variables can be picked up again and transformed. 

Known issues with sites:
- some domains may be hijacked (e.g. Poker domains show up in list)
- count every external link per domain only once
- some schools have several domains
- some schools have an updated website

Here are some examples of interesting sites:
- https://www.wien.gv.at/bildung/schulen/modeschule/kooperationen/forschungsprojekt-oeaw.html
- https://portal.billroth73.at
- https://eduthek.at
- https://www.fms15.at/impressum/


Some possible improvements: 
- Create iterative approach where those websites from comparison_df that have no data yet are scraped in a second step and the resulting data df has to be updated with new pages found. 

In [ ]:
## MASTER A website scraper that takes links as an input, visits sites, and 

input_df = comparison_df

import aiohttp
import asyncio
import time
from bs4 import BeautifulSoup
import pandas as pd
import json
from urllib.parse import urljoin, urlparse
from datetime import datetime


# Main async function to SCRAPE ALL URLs from a DataFrame
async def scrape_all(df):
    results = []
    domain_last_visited = {}
    
    async with aiohttp.ClientSession() as session:
        print("session: ")
        print(session)
        tasks = []
        for index, row in df.iterrows():
            base_url = row["updated_URL"]
            skz = row["SKZ"]
            
            if base_url in ["No URL", "Invalid URL"]:
                print(f"Skipping {skz}: {base_url}")
                continue

            print(f"Scheduling scrape for {base_url}...")

            task = asyncio.create_task(scrape_website(session, base_url, skz, domain_last_visited))
            tasks.append(task)
        
        results = await asyncio.gather(*tasks)
    
    return results

# Function to scrape a single website
async def scrape_website(session, base_url, skz, domain_last_visited):
    visited_internal_links = set()
    website_internal_links = set()
    website_all_links = set()
    attempted_visits_links = set()

    # Scrape the homepage
    base_url = base_url[:-1] if base_url.endswith("/") else base_url
    homepage_internal_links, homepage_all_links, visited_link = await scrape_links(session, base_url, domain_last_visited)
    attempted_visits_links.add(base_url)
    visited_internal_links.add(visited_link)
    # print(f"added {base_url} to visited_internal_links")
    website_internal_links.update(homepage_internal_links)
    website_all_links.update(homepage_all_links)

    # Scrape the internal links found on the homepage
    tasks = []
    for internal_link in homepage_internal_links:
        task = asyncio.create_task(scrape_links(session, internal_link, domain_last_visited))
        tasks.append(task)
        attempted_visits_links.add(internal_link)
        # print(f"added {internal_link} to attempted_visits_links")
        await asyncio.sleep(7)  # Ensure delay between internal page visits
    
    results = await asyncio.gather(*tasks)
    
    for internal_links, external_links, visited_internal_link in results:
        visited_internal_links.add(visited_internal_link)
        website_internal_links.update(internal_links)
        website_all_links.update(external_links)

    # # Here I would add the code for visiting the internal links found on the internal links
    # # Don't forget to change the filename of the output file
    # links_to_visit = website_internal_links - visited_internal_links
    # tasks = []
    # for internal_link in links_to_visit:
    #     task = asyncio.create_task(scrape_links(session, internal_link, domain_last_visited))
    #     tasks.append(task)
    #     attempted_visits_links.add(internal_link)
    #     print(f"added {internal_link} to attempted_visits_links")
    #     await asyncio.sleep(7)  # Ensure delay between internal page visits
    # results = await asyncio.gather(*tasks)
    # for internal_links, external_links, visited_internal_link in results:
    #     visited_internal_links.add(visited_internal_link)
    #     website_internal_links.update(internal_links)
    #     website_all_links.update(external_links)
    
    print(list(visited_internal_links))

    return {
        "SKZ": skz,
        "base_url": base_url,
        "internal_links": list(website_internal_links),
        "visited_internal_links": list(visited_internal_links),
        "attempted_visits_links": list(attempted_visits_links),
        "all_links": list(website_all_links),
    }

async def scrape_links(session, url, domain_last_visited):
    print(f"Visiting {url}")

    parsed_url = urlparse(url)
    base_url = parsed_url.scheme + "://" + parsed_url.netloc
    # from https to http, or vice versa
    base_url_alternative_scheme = base_url.replace("http://", "https://") if base_url.startswith("http://") else base_url.replace("https://", "http://")
    # from www to non-www, or vice versa
    base_url_www_switch = base_url
    base_url_alternative_scheme_www_switch = base_url_alternative_scheme

    if base_url.startswith("http://www.") or base_url.startswith("https://www."):
        base_url_www_switch = base_url.replace("www.", "")
        base_url_alternative_scheme_www_switch = base_url_alternative_scheme.replace("www.", "")
    elif base_url.startswith("http"):
        base_url_www_switch = base_url.replace("http://", "http://www.")
        base_url_alternative_scheme_www_switch = base_url.replace("https://", "https://www.")
    else:
        print(f"Invalid base_url: {base_url}")

    html = await fetch(session, url, domain_last_visited)

    if html:
            page_internal_links = set()
            page_all_links = set()
            visited_link = url
            print(f"added {url} to visited_internal_links")
            soup = BeautifulSoup(html, 'html.parser')
            for link in soup.find_all('a', href=True):
                link_href = link['href']
                full_url = urljoin(base_url, link_href)
                if not is_valid_url(full_url):
                    print(f"Skipping invalid URL: {full_url}")
                    continue
                page_all_links.add(full_url)
                if not (full_url.startswith(base_url) or full_url.startswith(base_url_alternative_scheme) or full_url.startswith(base_url_www_switch) or full_url.startswith(base_url_alternative_scheme_www_switch)):
                    continue
                if link_href.startswith(('javascript:', 'tel:', 'fax:', 'mailto:')) or any(link_href.endswith(ext) for ext in non_html_extensions):
                    continue
                clean = clean_url(full_url, base_url)
                page_internal_links.add(clean)
            return list(page_internal_links), list(page_all_links), visited_link
    return [], [], ""


# Function to remove URL fragments and queries
def clean_url(url, base_url):
    parsed = urlparse(url)
    netloc = parsed.netloc.split(':')[0]
    parsed = parsed._replace(netloc=netloc)
    #parsed = parsed._replace(query='')
    parsed = parsed._replace(fragment='')
    parsed = parsed._replace(path=parsed.path.rstrip('/'))
    parsed_base_url = urlparse(base_url)
    parsed = parsed._replace(scheme=parsed_base_url.scheme)
    parsed = parsed._replace(netloc=parsed_base_url.netloc)
    return parsed.geturl()

def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

non_html_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg', '.webp', '.ico', 
                       '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx', 
                       '.zip', '.rar', '.tar', '.gz', '.mp3', '.wav', '.mp4', '.avi']

# Function to FETCH and parse HTML content
async def fetch(session, url, domain_last_visited):
    domain = urlparse(url).netloc
    # Ensure at least 7 seconds between visits to the same domain
    if domain in domain_last_visited:
        elapsed_time = time.time() - domain_last_visited[domain]
        if elapsed_time < 7:
            await asyncio.sleep(7 - elapsed_time)
    try:
        async with session.get(url) as response:
            domain_last_visited[domain] = time.time()
            if response.status == 200:
                return await response.text()
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

# Function to save results to JSON
def save_results(results):
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = f"3_scraped_links_{date_str}.json"
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)
    print(f"Results saved to {filename}")

# Run the scraper
results = asyncio.run(scrape_all(input_df))

print(results)

# Save the results
save_results(results)



## Part 4: Cleaning and Transforming the data
We need to go through several steps: 
- Remove schools without a website
- remove duplicate websites
- Report on difference
- 

Known issues with sites:
- some domains may be hijacked (e.g. Poker domains show up in list)
- count every external link per domain only once
- some schools have several domains
- some schools have an updated website


In [ ]:
# 4.1 Comparative statistics about analysed data. 
# Loading of json file with links into a DataFrame
path = "./3_scraped_links_2024-08-12.json"
with open(path, 'r') as f:
    results = json.load(f)

# Create a DataFrame from the results
df_results = pd.DataFrame(results)

df_results.head()



In [ ]:
## Example data
{
    "SKZ": "922012",
    "base_url": "https://gts-anton-sattler-gasse.schule.wien.at",
    "internal_links": [
        "https://gts-anton-sattler-gasse.schule.wien.at/datenschutz",
        "https://gts-anton-sattler-gasse.schule.wien.at/administration",
        "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
        "https://gts-anton-sattler-gasse.schule.wien.at/elternverein",
    ],
    "visited_internal_links": [
        "https://gts-anton-sattler-gasse.schule.wien.at/datenschutz",
        "",
        "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
    ],
    "attempted_visits_links": [
        "https://gts-anton-sattler-gasse.schule.wien.at/datenschutz",
        "https://gts-anton-sattler-gasse.schule.wien.at/administration",
        "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
    ],
    "all_links": [
        "http://www.lehrerweb.at",
        "https://youtu.be/tuvt4R3ozg8",
        "https://gts-anton-sattler-gasse.schule.wien.at/administration",
        "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
        "https://gts-anton-sattler-gasse.schule.wien.at/elternverein",
        "https://gts-anton-sattler-gasse.schule.wien.at/puma",
        "https://gts-anton-sattler-gasse.schule.wien.at/about",
        "https://www.schule.wien.at/kontaktformular-fuer-e-mails/?no_cache=1&tx_email2powermail%5Bid%5D=327",
        "http://puma.lehrerweb.at",
        "https://gts-anton-sattler-gasse.schule.wien.at/",
        "https://gts-anton-sattler-gasse.schule.wien.at",
        "https://www.bmbwf.gv.at/Themen/schule/schulrecht/ds.html",
        "https://privacy.microsoft.com/de-de/privacystatement",
    ]
},

In [ ]:
import pandas as pd

# Creating DataFrames
links_df = df_results
schools_df = comparison_df

# Convert 'SKZ' to string for proper merging
links_df['SKZ'] = links_df['SKZ'].astype(str)
schools_df['SKZ'] = schools_df['SKZ'].astype(str)

# Total Number of Websites
total_websites = links_df['SKZ'].nunique()
print(f"Total Number of Websites: {total_websites}")

# Websites with Data vs. No Data
links_df['has_data'] = links_df['all_links'].apply(lambda x: len(x) > 0)
websites_with_data = links_df['has_data'].sum()
websites_without_data = len(links_df) - websites_with_data
print(f"Websites with Data: {websites_with_data}")
print(f"Websites without Data: {websites_without_data}")

# Total Number of Links
total_links = links_df['all_links'].apply(len).sum()
print(f"Total Number of Links: {total_links}")

# Internal vs. External Links
total_internal_links = links_df['internal_links'].apply(len).sum()
total_external_links = total_links - total_internal_links
print(f"Total Internal Links: {total_internal_links}")
print(f"Total External Links: {total_external_links}")

# Average Number of Links per Website
average_total_links_per_website = links_df['all_links'].apply(len).mean()
average_internal_links_per_website = links_df['internal_links'].apply(len).mean()
average_external_links_per_website = average_total_links_per_website - average_internal_links_per_website
print(f"Average Number of Total Links per Website: {average_total_links_per_website:.2f}")
print(f"Average Number of Internal Links per Website: {average_internal_links_per_website:.2f}")
print(f"Average Number of External Links per Website: {average_external_links_per_website:.2f}")

# Merge DataFrames to analyze by school type
merged_df = pd.merge(links_df, schools_df, on='SKZ', how='left')

# Analysis by School Type
school_type_group = merged_df.groupby('school_type_txt')

# Aggregate functions for groupby
def total_links_func(x):
    return sum(len(links) for links in x)

def average_links_func(x):
    return sum(len(links) for links in x) / len(x)

# Compute aggregated statistics
school_type_analysis = school_type_group.agg(
    total_websites=('SKZ', 'nunique'),
    websites_with_data=('has_data', 'sum'),
    websites_without_data=('has_data', lambda x: len(x) - x.sum()),
    total_links=('all_links', total_links_func),
    total_internal_links=('internal_links', total_links_func),
    average_total_links_per_website=('all_links', average_links_func),
    average_internal_links_per_website=('internal_links', average_links_func)
)

# Calculate total external links and average external links per website
school_type_analysis['total_external_links'] = school_type_analysis['total_links'] - school_type_analysis['total_internal_links']
school_type_analysis['average_external_links_per_website'] = school_type_analysis['average_total_links_per_website'] - school_type_analysis['average_internal_links_per_website']

print(school_type_analysis)


In [ ]:
# Cleaning the links

# open the json file with the links as a dataframe
import pandas as pd
import json
from datetime import datetime

# Load the JSON file into a DataFrame
path = "./scraped_links_2024-08-12.json"
with open(path, 'r') as f:
    results = json.load(f)

# Create a DataFrame from the results
links_df = pd.DataFrame(results)

# Remove unwanted links and clean the remaining links

{
        "SKZ": "922012",
        "base_url": "https://gts-anton-sattler-gasse.schule.wien.at",
        "internal_links": [
            "https://gts-anton-sattler-gasse.schule.wien.at/datenschutz",
            "https://gts-anton-sattler-gasse.schule.wien.at/administration",
            "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
            "https://gts-anton-sattler-gasse.schule.wien.at/elternverein",
            "https://gts-anton-sattler-gasse.schule.wien.at/puma",
        ],
        "visited_internal_links": [
            "https://gts-anton-sattler-gasse.schule.wien.at/datenschutz",
            "",
            "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",

        ],
        "attempted_visits_links": [
            "https://gts-anton-sattler-gasse.schule.wien.at/datenschutz",
            "https://gts-anton-sattler-gasse.schule.wien.at/administration",
            "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
            "https://gts-anton-sattler-gasse.schule.wien.at/elternverein",

        ],
        "all_links": [
            "http://www.lehrerweb.at",
            "https://youtu.be/tuvt4R3ozg8",
            "https://gts-anton-sattler-gasse.schule.wien.at/administration",
            "https://gts-anton-sattler-gasse.schule.wien.at/schwerpunkte",
            "https://gts-anton-sattler-gasse.schule.wien.at/elternverein",
            "https://gts-anton-sattler-gasse.schule.wien.at/puma",

        ]
    },

# Remove internal links based on stripped 'base_url'


output_data = []

for i, row in links_df.iterrows():
    parsed_base_url = urlparse(row['base_url'])
    stripped_base_url = parsed_base_url.netloc.replace("www.", "")
    # stripped_base_url = row['base_url'].replace("http://", "").replace("https://", "").rstrip('/').replace("www.", "")
    # Filter out faulty links and internal links
    cleaned_links = [link for link in row['all_links'] 
                     if not link.strip().lower().startswith((f"http://{stripped_base_url.lower()}", 
                                                             f"https://{stripped_base_url.lower()}", 
                                                             f"http://www.{stripped_base_url.lower()}", 
                                                             f"https://www.{stripped_base_url.lower()}", 
                                                             "javascript:", "tel:", "fax:", "mailto:")) 
                     and " " not in link]    # Remove links with spaces
    cleaned_links = [link.replace('http://', '').replace('https://', '').replace('www.', '').split('#')[0].split('?')[0].rstrip('/') for link in cleaned_links]
    cleaned_domains = [link.split("/", 1)[0].lower() for link in cleaned_links]
    for i, domain in enumerate(cleaned_domains):
        # merge social media domains
        # youtu.be -> youtube.com
        if domain == "youtu.be":
            cleaned_domains[i] = "youtube.com"
        # de-de.facebook.com -> facebook.com
        elif domain.endswith(".facebook.com"):
            cleaned_domains[i] = "facebook.com"
        elif domain.endswith("fb.com"):
            cleaned_domains[i] = "facebook.com"
        # vm.tiktok.com -> tiktok.com
        elif domain.endswith(".tiktok.com"):
            cleaned_domains[i] = "tiktok.com"
        # at.linkedin.com -> linkedin.com
        elif domain.endswith(".linkedin.com"):
            cleaned_domains[i] = "linkedin.com"
        # de.wikipedia.org -> wikipedia.org
        elif domain.endswith(".wikipedia.org"):
            cleaned_domains[i] = "wikipedia.org"
        if domain.endswith("de.wordpress.org"):
            cleaned_domains[i] = "wordpress.org"


    # Remove duplicates and sort
    cleaned_links = sorted(set(cleaned_links))
    cleaned_domains = sorted(set(cleaned_domains))
    

    # Construct the output dictionary entry for this row
    entry = {
        "SKZ": row['SKZ'],
        "base_url": row['base_url'],
        # "stripped_base_url": stripped_base_url,
        # "internal_links": row['internal_links'],
        # "visited_internal_links": row['visited_internal_links'],
        # "attempted_visits_links": row['attempted_visits_links'],
        # "all_links": row['all_links'],
        "cleaned_ext_links": cleaned_links,
        "cleaned_ext_domains": cleaned_domains,
        "count_ext_links": len(cleaned_links),
        "count_ext_domains": len(cleaned_domains),
        "count_int_links": len(row['internal_links']),
        "count_attempted_links": len(row['attempted_visits_links']),
        "count_visited_links": len(row['visited_internal_links']),
    }
    
    output_data.append(entry)

# save the cleaned links to a new json file with a filename that's based on the original path
path_cleaned = path.replace(".json", "_cleaned.json")
with open(path_cleaned, 'w') as f:
    json.dump(output_data, f, indent=4)
print(f"Cleaned links saved to {path_cleaned}")










In [ ]:
# Descriptive stats of internal links by school_type_mapping, without schools that have 0 links. 

import pandas as pd
import json
import numpy as np

# Load the school type CSV file
csv_path = './comparison_df_2024-08-12.csv'
comparison_df = pd.read_csv(csv_path)

# Load the scraped links JSON file
file_path = 'scraped_links_2024-08-12_cleaned.json'
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Convert JSON data to DataFrame
links_df = pd.DataFrame(schools_data)

# Merge the two DataFrames on 'SKZ'
merged_df = comparison_df.merge(links_df, on='SKZ', how='left')

# Filter out schools with count_int_links == 0
filtered_df = merged_df[merged_df['count_int_links'] > 0]

# Group by 'school_type_mapping' and calculate statistics for 'count_int_links'
statistics_df = filtered_df.groupby('school_type_mapping').agg({
    'count_int_links': ['mean', 'std', 'median', 'min', 'max', 'count']
}).reset_index()

# Flatten multi-level columns
statistics_df.columns = ['School Type Mapping', 'Mean', 'Std Dev', 'Median', 'Min', 'Max', 'N']

# Calculate overall statistics for all schools with non-zero internal links
overall_stats = filtered_df['count_int_links'].agg(['mean', 'std', 'median', 'min', 'max', 'count'])
overall_row = pd.DataFrame([[
    'Total',
    overall_stats['count'],
    overall_stats['mean'],
    overall_stats['std'],
    overall_stats['median'],
    overall_stats['min'],
    overall_stats['max']
]], columns=['School Type Mapping', 'N', 'Mean', 'Std Dev', 'Median', 'Min', 'Max'])

# Append the 'Total' row to the statistics DataFrame
statistics_df = pd.concat([statistics_df, overall_row], ignore_index=True)

# Reorder columns
statistics_df = statistics_df[['School Type Mapping', 'N', 'Mean', 'Std Dev', 'Median', 'Min', 'Max']]

# Print LaTeX table code
latex_code = '''
\\begin{table}[htbp]
\\centering
\\resizebox{\\textwidth}{!}{
    \\begin{tabular}{lcccccc}
    \\toprule
    School Type Mapping & N & Mean & Std Dev & Median & Min & Max \\\\
    \\midrule
'''

for index, row in statistics_df.iterrows():
    latex_code += f"    {row['School Type Mapping']} & {int(row['N'])} & {row['Mean']:.2f} & {row['Std Dev']:.2f} & {row['Median']:.2f} & {row['Min']:.2f} & {row['Max']:.2f} \\\\\n"

latex_code += '''
    \\bottomrule
    \\end{tabular}
}
\\caption{Statistics of Internal Links by School Type Mapping}
\\label{tab:internal_links_stats_mapping}
\\end{table}
'''

print(latex_code)
#  Don't forget to add a \midrule before the total row. 


In [ ]:
# Comparison of Internal Sites Discovered, Attempted, and Visited by School Type Mapping

import pandas as pd
import json
import numpy as np

# Load the school type CSV file
csv_path = './comparison_df_2024-08-12.csv'
comparison_df = pd.read_csv(csv_path)

# Load the scraped links JSON file
file_path = 'scraped_links_2024-08-12_cleaned.json'
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Convert JSON data to DataFrame
links_df = pd.DataFrame(schools_data)

# Merge the two DataFrames on 'SKZ'
merged_df = comparison_df.merge(links_df, on='SKZ', how='left')

# Filter out schools with count_int_links == 0
filtered_df = merged_df[merged_df['count_int_links'] > 0]

# Group by 'school_type_mapping' and calculate statistics for 'count_int_links', 'count_attempted_links', and 'count_visited_links'
statistics_df = filtered_df.groupby('school_type_mapping').agg({
    'count_int_links': ['mean', 'std', 'count'],
    'count_attempted_links': ['mean', 'std'],
    'count_visited_links': ['mean', 'std']
}).reset_index()

# Flatten multi-level columns
statistics_df.columns = ['School Type Mapping', 'N_Discovered_Mean', 'N_Discovered_SD', 'N_Discovered_Count',
                          'Attempted_Mean', 'Attempted_SD', 'Visited_Mean', 'Visited_SD']

# Calculate overall statistics for all schools with non-zero internal links
overall_stats = filtered_df.agg({
    'count_int_links': ['mean', 'std', 'count'],
    'count_attempted_links': ['mean', 'std'],
    'count_visited_links': ['mean', 'std']
})
overall_row = pd.DataFrame([[
    'Total',
    overall_stats['count_int_links']['mean'],
    overall_stats['count_int_links']['std'],
    overall_stats['count_int_links']['count'],
    overall_stats['count_attempted_links']['mean'],
    overall_stats['count_attempted_links']['std'],
    overall_stats['count_visited_links']['mean'],
    overall_stats['count_visited_links']['std']
]], columns=['School Type Mapping', 'N_Discovered_Mean', 'N_Discovered_SD', 'N_Discovered_Count',
             'Attempted_Mean', 'Attempted_SD', 'Visited_Mean', 'Visited_SD'])

# Append the 'Total' row to the statistics DataFrame
statistics_df = pd.concat([statistics_df, overall_row], ignore_index=True)

# Calculate the total numbers of discovered, attempted, and visited links
total_N = int(merged_df['count_int_links'].count())
total_discovered = int(merged_df['count_int_links'].sum())
total_attempted = int(merged_df['count_attempted_links'].sum())
total_visited = int(merged_df['count_visited_links'].sum())

# Print LaTeX table code
latex_code = '''
\\begin{table}[htbp]
\\centering
\\resizebox{\\textwidth}{!}{
    \\begin{tabular}{lccc*{2}{cc}*{2}{cc}}
    \\toprule
    \\multirow{2}{*}{\\makecell[l]{School Type}} & \\multirow{2}{*}{n} & \\multicolumn{2}{c}{Internal} & \\multicolumn{2}{c}{Attempted} & \\multicolumn{2}{c}{Visited} \\\\
    \\cmidrule(lr){3-4} \\cmidrule(lr){5-6} \\cmidrule(lr){7-8}
    & & Avg. & SD & Avg. & SD & Avg. & SD \\\\
    \\midrule
'''

for index, row in statistics_df.iterrows():
    latex_code += f"    {row['School Type Mapping']} & {int(row['N_Discovered_Count'])} & {row['N_Discovered_Mean']:.2f} & {row['N_Discovered_SD']:.2f} & {row['Attempted_Mean']:.2f} & {row['Attempted_SD']:.2f} & {row['Visited_Mean']:.2f} & {row['Visited_SD']:.2f} \\\\\n"

latex_code += '''
    \\midrule
    Absolute n of webpages discovered / visited & {} & \\multicolumn{{2}}{{c}}{{{:,}}} & \\multicolumn{{2}}{{c}}{{{:,}}} & \\multicolumn{{2}}{{c}}{{{:,}}} \\\\
'''.format(total_N, total_discovered, total_attempted, total_visited)

latex_code += '''
    \\bottomrule
    \\end{tabular}
}
\\caption{Website Visits Compared: Internal Links Discovered, Attempted to Visit, and Actually Visited}
\\label{tab:sites_comparison}
\\end{table}
'''

print(latex_code)


In [ ]:
# External links with domains by school_type_mapping, without schools that have 0 links.

import pandas as pd
import json
import numpy as np

# Load the school type CSV file
csv_path = './comparison_df_2024-08-12.csv'
comparison_df = pd.read_csv(csv_path)

# Load the scraped links JSON file
file_path = 'scraped_links_2024-08-12_cleaned.json'
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Convert JSON data to DataFrame
links_df = pd.DataFrame(schools_data)

# Merge the two DataFrames on 'SKZ'
merged_df = comparison_df.merge(links_df, on='SKZ', how='left')

# Filter out schools with count_int_links == 0
filtered_df = merged_df[merged_df['count_int_links'] > 0]

# Group by 'school_type_mapping' and calculate statistics for 'count_ext_links' and 'count_ext_domains'
statistics_df = filtered_df.groupby('school_type_mapping').agg({
    'count_ext_links': ['mean', 'std', 'median', 'count'],
    'count_ext_domains': ['mean', 'std', 'median']
}).reset_index()

# Flatten multi-level columns
statistics_df.columns = ['School Type Mapping', 'Ext. Links Mean', 'Ext. Links SD', 'Ext. Links Median', 'N', 'Ext. Domains Mean', 'Ext. Domains SD', 'Ext. Domains Median']

# Calculate overall statistics for all schools with non-zero internal links
overall_stats = filtered_df.agg({
    'count_ext_links': ['mean', 'std', 'median', 'count'],
    'count_ext_domains': ['mean', 'std', 'median']
})
overall_row = pd.DataFrame([[
    'Total',
    overall_stats['count_ext_links']['mean'],
    overall_stats['count_ext_links']['std'],
    overall_stats['count_ext_links']['median'],
    overall_stats['count_ext_links']['count'],
    overall_stats['count_ext_domains']['mean'],
    overall_stats['count_ext_domains']['std'],
    overall_stats['count_ext_domains']['median']
]], columns=['School Type Mapping', 'Ext. Links Mean', 'Ext. Links SD', 'Ext. Links Median', 'N', 'Ext. Domains Mean', 'Ext. Domains SD', 'Ext. Domains Median'])

# Append the 'Total' row to the statistics DataFrame
statistics_df = pd.concat([statistics_df, overall_row], ignore_index=True)

# Print LaTeX table code
latex_code = '''
\\begin{table}[htbp]
\\centering
\\resizebox{\\textwidth}{!}{
    \\begin{tabular}{lccc*{3}{cc}}
    \\toprule
    \\multirow{2}{*}{\\makecell[l]{School Type}} & \\multirow{2}{*}{n} & \\multicolumn{3}{c}{Ext. Links} & \\multicolumn{3}{c}{Ext. Domains} \\\\
    \\cmidrule(lr){3-5} \\cmidrule(lr){6-8}
    & & Avg. & SD & Median & Avg. & SD & Median \\\\
    \\midrule
'''

for index, row in statistics_df.iterrows():
    latex_code += f"    {row['School Type Mapping']} & {int(row['N'])} & {row['Ext. Links Mean']:.2f} & {row['Ext. Links SD']:.2f} & {row['Ext. Links Median']:.2f} & {row['Ext. Domains Mean']:.2f} & {row['Ext. Domains SD']:.2f} & {row['Ext. Domains Median']:.2f} \\\\\n"

latex_code += '''
    \\bottomrule
    \\end{tabular}
}
\\caption{Comparison of External Links and Domains by School Type Mapping}
\\label{tab:external_links_domains}
\\end{table}
'''

print(latex_code)


In [ ]:
# External links with domains by school_type_text, with median and total, without schools that have 0 links.
import pandas as pd
import json
import numpy as np

# Load the school type CSV file
csv_path = './comparison_df_2024-08-12.csv'
comparison_df = pd.read_csv(csv_path)

# Load the scraped links JSON file
file_path = 'scraped_links_2024-08-12_cleaned.json'
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Convert JSON data to DataFrame
links_df = pd.DataFrame(schools_data)

# Merge the two DataFrames on 'SKZ'
merged_df = comparison_df.merge(links_df, on='SKZ', how='left')

# Filter out schools with count_int_links == 0
filtered_df = merged_df[merged_df['count_int_links'] > 0]

# Group by 'school_type_txt' and calculate statistics for 'count_ext_links' and 'count_ext_domains'
statistics_df = filtered_df.groupby('school_type_txt').agg({
    'count_ext_links': ['mean', 'std', 'median', 'count'],
    'count_ext_domains': ['mean', 'std', 'median']
}).reset_index()

# Flatten multi-level columns
statistics_df.columns = ['School Type', 'Ext. Links Mean', 'Ext. Links SD', 'Ext. Links Median', 'N', 'Ext. Domains Mean', 'Ext. Domains SD', 'Ext. Domains Median']

# Calculate overall statistics for all schools with non-zero internal links
overall_stats = filtered_df.agg({
    'count_ext_links': ['mean', 'std', 'median', 'count'],
    'count_ext_domains': ['mean', 'std', 'median']
})
overall_row = pd.DataFrame([[
    'Total',
    overall_stats['count_ext_links']['mean'],
    overall_stats['count_ext_links']['std'],
    overall_stats['count_ext_links']['median'],
    overall_stats['count_ext_links']['count'],
    overall_stats['count_ext_domains']['mean'],
    overall_stats['count_ext_domains']['std'],
    overall_stats['count_ext_domains']['median']
]], columns=['School Type', 'Ext. Links Mean', 'Ext. Links SD', 'Ext. Links Median', 'N', 'Ext. Domains Mean', 'Ext. Domains SD', 'Ext. Domains Median'])

# Append the 'Total' row to the statistics DataFrame
statistics_df = pd.concat([statistics_df, overall_row], ignore_index=True)

# Print LaTeX table code
latex_code = '''
\\begin{table}[htbp]
\\centering
\\resizebox{\\textwidth}{!}{
    \\begin{tabular}{lccc*{3}{cc}}
    \\toprule
    \\multirow{2}{*}{\\makecell[l]{School Type}} & \\multirow{2}{*}{n} & \\multicolumn{3}{c}{Ext. Links} & \\multicolumn{3}{c}{Ext. Domains} \\\\
    \\cmidrule(lr){3-5} \\cmidrule(lr){6-8}
    & & Avg. & SD & Median & Avg. & SD & Median \\\\
    \\midrule
'''

for index, row in statistics_df.iterrows():
    latex_code += f"    {row['School Type']} & {int(row['N'])} & {row['Ext. Links Mean']:.2f} & {row['Ext. Links SD']:.2f} & {row['Ext. Links Median']:.2f} & {row['Ext. Domains Mean']:.2f} & {row['Ext. Domains SD']:.2f} & {row['Ext. Domains Median']:.2f} \\\\\n"

latex_code += '''
    \\bottomrule
    \\end{tabular}
}
\\caption{Comparison of External Links and Domains by School Type}
\\label{tab:external_links_domains_txt}
\\end{table}
'''

print(latex_code)


In [ ]:
# Comparison of External Links and Domains by School Type (needs manual adding of two \ to )

import pandas as pd
import json
import numpy as np

# Load the school type CSV file
csv_path = './comparison_df_2024-08-12.csv'
comparison_df = pd.read_csv(csv_path)

# Load the scraped links JSON file
file_path = 'scraped_links_2024-08-12_cleaned.json'
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Convert JSON data to DataFrame
links_df = pd.DataFrame(schools_data)

# Merge the two DataFrames on 'SKZ'
merged_df = comparison_df.merge(links_df, on='SKZ', how='left')

# Filter out schools with count_int_links == 0
filtered_df = merged_df[merged_df['count_int_links'] > 0]

# Group by 'school_type_txt' and calculate statistics for 'count_ext_links' and 'count_ext_domains'
statistics_df = filtered_df.groupby('school_type_txt').agg({
    'count_ext_links': ['mean', 'std', 'count'],
    'count_ext_domains': ['mean', 'std']
}).reset_index()

# Flatten multi-level columns
statistics_df.columns = ['School Type', 'Ext. Links Mean', 'Ext. Links SD', 'N', 'Ext. Domains Mean', 'Ext. Domains SD']

# Calculate overall statistics for all schools with non-zero internal links
overall_stats = filtered_df.agg({
    'count_ext_links': ['mean', 'std', 'count'],
    'count_ext_domains': ['mean', 'std']
})
overall_row = pd.DataFrame([[
    'Total',
    overall_stats['count_ext_links']['mean'],
    overall_stats['count_ext_links']['std'],
    overall_stats['count_ext_links']['count'],
    overall_stats['count_ext_domains']['mean'],
    overall_stats['count_ext_domains']['std']
]], columns=['School Type', 'Ext. Links Mean', 'Ext. Links SD', 'N', 'Ext. Domains Mean', 'Ext. Domains SD'])

# Append the 'Total' row to the statistics DataFrame
statistics_df = pd.concat([statistics_df, overall_row], ignore_index=True)

# Print LaTeX table code
latex_code = '''
\\begin{table}[htbp]
\\centering
\\resizebox{\\textwidth}{!}{
    \\begin{tabular}{lccc*{2}{cc}}
    \\toprule
    \\multirow{2}{*}{\\makecell[l]{School Type}} & \\multirow{2}{*}{n} & \\multicolumn{2}{c}{Ext. Links} & \\multicolumn{2}{c}{Ext. Domains} \\
    \\cmidrule(lr){3-4} \\cmidrule(lr){5-6}
    & & Avg. & SD & Avg. & SD \\
    \\midrule
'''

for index, row in statistics_df.iterrows():
    latex_code += f"    {row['School Type']} & {int(row['N'])} & {row['Ext. Links Mean']:.2f} & {row['Ext. Links SD']:.2f} & {row['Ext. Domains Mean']:.2f} & {row['Ext. Domains SD']:.2f} \\\\\n"

latex_code += '''
    \\bottomrule
    \\end{tabular}
}
\\caption{Comparison of External Links and Domains by School Type}
\\label{tab:external_links_domains_txt_no_median}
\\end{table}
'''

print(latex_code)


In [ ]:
import pandas as pd
import json
import numpy as np

# Load the comparison dataframe
csv_path = './comparison_df_2024-08-12.csv'
comparison_df = pd.read_csv(csv_path)

# Load the JSON file with scraped links data
file_path = 'scraped_links_2024-08-12_cleaned.json'
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Convert the JSON data to a dataframe
scraped_df = pd.DataFrame(schools_data)

# Merge the comparison dataframe with scraped data based on 'SKZ'
merged_df = pd.merge(comparison_df[['SKZ', 'school_type_mapping']], scraped_df, on='SKZ')

# Group by 'school_type_mapping' and calculate the statistics
grouped_stats = merged_df.groupby('school_type_mapping').agg(
    avg_int_links=('count_int_links', 'mean'),
    std_int_links=('count_int_links', 'std'),
    median_int_links=('count_int_links', 'median'),
    min_int_links=('count_int_links', 'min'),
    max_int_links=('count_int_links', 'max'),
    avg_attempted_links=('count_attempted_links', 'mean'),
    std_attempted_links=('count_attempted_links', 'std'),
    avg_visited_links=('count_visited_links', 'mean'),
    std_visited_links=('count_visited_links', 'std')
).reset_index()

# Generate LaTeX table for internal links statistics
latex_table_1 = grouped_stats[['school_type_mapping', 'avg_int_links', 'std_int_links', 'median_int_links', 
                               'min_int_links', 'max_int_links']].to_latex(index=False, 
                               header=["School Type", "Average", "Standard Deviation", "Median", "Min", "Max"], 
                               column_format="lccccc", 
                               float_format="%.2f", 
                               caption="Internal Links Statistics by School Type", 
                               label="tab:internal_links_stats")

# Generate LaTeX table comparing internal, attempted, and visited links
latex_table_2 = grouped_stats[['school_type_mapping', 'avg_int_links', 'std_int_links', 
                               'avg_attempted_links', 'std_attempted_links', 
                               'avg_visited_links', 'std_visited_links']].to_latex(index=False, 
                               header=["School Type", "Internal Links", "", "Attempted Links", "", "Visited Links", ""], 
                               column_format="lcccccc", 
                               float_format="%.2f", 
                               multicolumn=True, 
                               multicolumn_format="c", 
                               caption="Comparison of Internal, Attempted, and Visited Links by School Type", 
                               label="tab:link_comparison")

print(latex_table_1)
print(latex_table_2)

In [ ]:
import json
from collections import Counter
from urllib.parse import urlparse

# Step 1: Load the JSON file
file_path = 'scraped_links_2024-08-12_cleaned.json'  # Replace with the path to your JSON file
with open(file_path, 'r') as file:
    schools_data = json.load(file)

# Step 2: Extract the cleaned_domains
cleaned_domains = []
for school in schools_data:
    cleaned_domains.extend(school.get('cleaned_ext_domains', []))

# Step 3: Extract the TLD from each domain
tlds = []
for domain in cleaned_domains:
    parsed_url = urlparse(f'http://{domain}')  # Ensure URL is parsed correctly
    tld = parsed_url.hostname.split('.')[-1]  # Extract TLD
    tlds.append(tld)

# Step 4: Count the frequencies of each TLD
tld_counts = Counter(tlds)

# Step 5: Sort TLDs by the length of the TLD string
sorted_tlds_by_length = sorted(tld_counts.items(), key=lambda item: len(item[0]))

# Display the result
for tld, count in sorted_tlds_by_length:
    print(f"{tld}: {count}")


# 5. Categorization and Summarization
This scraper will categorize all external domains. 

For this purpose it will go through several steps: 
1. Visit each domain for the first 400 characters
2. It will use a GPT model to summarize what they are doing
3. Based on the summary it will categorize the organization based on 13 pre-defined categories

In case a website cannot be reached, a follow-up code block can re-attempt only those actors that are missing in the previous output file. 

Summaries are done in German and English.

Possible improvements: 
- Clearer Separation of steps, currently several approaches were compared and tested and old artifacts are still present. 
- Simpler testing for clarity. Manual checks for obvious miscategorisations were necessary because sometimes multiple categories may fit. 
- Clearer iterative code for those website not reached yet. First try all, then try those missing

Main learnings: 
- When using sample input and output, the categorization will be biased towards the provided sample categories. Therefore it is better to just give more generic prompts and no eample output data. 



In [ ]:
import nest_asyncio
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Function to format URLs
def format_url(url):
    if not url.startswith(('http://', 'https://')):
        url = 'http://' + url
    return url

# Function to clean text
def clean_text(text):
    return ' '.join(text.split())

# Function to try different URL formats
async def try_different_formats(session, url, headers):
    formats = [
        url,
        url.replace('http://', 'https://'),
        'http://www.' + urlparse(url).netloc,
        'https://www.' + urlparse(url).netloc
    ]
    for fmt in formats:
        try:
            async with session.get(fmt, headers=headers, timeout=10) as response:
                if response.status == 200:
                    html = await response.text()
                    soup = BeautifulSoup(html, 'html.parser')
                    text = soup.get_text()
                    cleaned_text = clean_text(text[:600])  # Clean the text
                    return fmt, cleaned_text
        except Exception:
            time.sleep(1)  # Introduce a delay before retrying
            continue
    return url, "Connection failed: All formats tried"

# Function to fetch and parse a single URL
async def fetch(session, url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }
    url = format_url(url)  # Format the URL
    try:
        async with session.get(url, headers=headers, timeout=15) as response:
            if response.status == 200:
                html = await response.text()
                soup = BeautifulSoup(html, 'html.parser')
                text = soup.get_text()
                cleaned_text = clean_text(text[:600])  # Clean the text
                return url, cleaned_text  # Return the URL and the cleaned text
            else:
                return await try_different_formats(session, url, headers)
    except aiohttp.ClientError:
        return await try_different_formats(session, url, headers)
    except Exception as e:
        return url, f"Connection failed: {str(e)}"

# Function to handle fetching multiple URLs
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            task = asyncio.ensure_future(fetch(session, url))
            tasks.append(task)
            await asyncio.sleep(0.1)  # Introduce a small delay after each request
        responses = await asyncio.gather(*tasks)
        return responses

# List of 5000 URLs to visit
urls = unique_domains

# Run the scraper
responses = asyncio.run(fetch_all(urls))

# Convert results to a DataFrame
df = pd.DataFrame(responses, columns=["Domain", "Text"])

# Evaluate the results
successful_connections = df[df["Text"].str.contains("Connection failed") == False].shape[0]
failed_connections = df[df["Text"].str.contains("Connection failed")].shape[0]

print(f"Successful connections: {successful_connections}")
print(f"Failed connections: {failed_connections}")

# Display the first few rows of the dataframe
print(df.head())

# Save the results to a CSV file
df.to_csv("5_scraped_data_round2.csv", index=False)

In [ ]:
# drop those where the text has less than 80 characters
df = df[df['Text'].str.len() > 80]

In [ ]:
# 5.1.1 Try again with those where connection has failed. 

import nest_asyncio
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Function to format URLs
def format_url(url):
    if not url.startswith(('http://', 'https://')):
        url = 'http://' + url
    return url

# Function to clean text
def clean_text(text):
    return ' '.join(text.split())

# Function to try different URL formats
async def try_different_formats(session, url, headers):
    formats = [
        url,
        url.replace('http://', 'https://'),
        'http://www.' + urlparse(url).netloc,
        'https://www.' + urlparse(url).netloc
    ]
    for fmt in formats:
        try:
            async with session.get(fmt, headers=headers, timeout=10) as response:
                if response.status == 200:
                    html = await response.text()
                    soup = BeautifulSoup(html, 'html.parser')
                    text = soup.get_text()
                    cleaned_text = clean_text(text[:600])  # Clean the text
                    return fmt, cleaned_text
        except Exception:
            time.sleep(1)  # Introduce a delay before retrying
            continue
    return url, "Connection failed: All formats tried"

# Function to fetch and parse a single URL
async def fetch(session, url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }
    url = format_url(url)  # Format the URL
    try:
        async with session.get(url, headers=headers, timeout=10) as response:
            if response.status == 200:
                html = await response.text()
                soup = BeautifulSoup(html, 'html.parser')
                text = soup.get_text()
                cleaned_text = clean_text(text[:600])  # Clean the text
                return url, cleaned_text  # Return the URL and the cleaned text
            else:
                return await try_different_formats(session, url, headers)
    except aiohttp.ClientError:
        return await try_different_formats(session, url, headers)
    except Exception as e:
        return url, f"Connection failed: {str(e)}"

# Function to handle fetching multiple URLs
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            task = asyncio.create_task(fetch(session, url))
            tasks.append(task)
            await asyncio.sleep(0.3)  # Introduce a small delay after each request
        responses = await asyncio.gather(*tasks)
        return responses

# Load the previously saved DataFrame
df = pd.read_csv("scraped_data.csv")

# Filter out the URLs that had connection failures
failed_urls = df[df["Text"].str.contains("Connection failed", na=False)]["Domain"].tolist()

# Retry the failed URLs
if failed_urls:
    retry_responses = asyncio.run(fetch_all(failed_urls))

    # Convert retry results to a DataFrame
    retry_df = pd.DataFrame(retry_responses, columns=["Domain", "Text"])

    # Update the original DataFrame with the retry results
    for index, row in retry_df.iterrows():
        df.loc[df["Domain"] == row["Domain"], "Text"] = row["Text"]

    # Save the updated results to a CSV file
    df.to_csv("scraped_data_updated.csv", index=False)

    # Evaluate the results
    successful_connections = df[df["Text"].str.contains("Connection failed", na=False) == False].shape[0]
    failed_connections = df[df["Text"].str.contains("Connection failed", na=False)].shape[0]

    print(f"Successful connections after retry: {successful_connections}")
    print(f"Failed connections after retry: {failed_connections}")

    # Display the first few rows of the updated dataframe
    print(df.head())
else:
    print("No failed connections to retry.")

In [ ]:
import pandas as pd

# Load CSV
# df = pd.read_csv("scraped_data_round2.csv")

# Filter out rows with "Connection failed"
df_filtered = df[~df["Text"].str.contains("Connection failed", na=False)]

# Remove the http:// from the domain
df_filtered["Domain"] = df_filtered["Domain"].str.replace("http://", "", regex=False)

# Also remove the www. from the domain
df_filtered["Domain"] = df_filtered["Domain"].str.replace("www.", "", regex=False)

# and remove the nan entries
df_filtered = df_filtered.dropna()

# Convert to dictionary
df_dict = df_filtered.set_index("Domain")["Text"].to_dict()

# Print the dictionary line by line
print("Final dictionary:")
for key, value in df_dict.items():
    print(f"{key}: {value}")

# Count the total number of entries
total_entries = len(df_dict)
print(f"\nTotal number of entries: {total_entries}")

In [ ]:
# 5.2 Automatic Summary of the text for each domain with GPT-4o-mini
## Q: Which language to use for the summary? Original German or English? Or both?
summary_prompt = """
You are a summariser of websites and only output JSON You have been provided with a list of URLs and the first 600 characters of text from each URL.
- Please provide two summariesa summary of the text for each URL, one in German and one in English.
- The summary should be concise and informative about the type of organization and what activities they might provide to schools and teaches.
- You can also use what you already know about the organization from the URL.
- Don't use any PII in the summaries.
- If there is no text available, please leave it empty or provide a minimal summary if you know something about the URL."""

sum_input_example = """{{"example.com": 'This is the website text from example.com', 'example2.com': 'This is the website text from example2.com'}"""
sum_output_example = """{"example.com": {
   "summary_en": "English Summary",
   "summary_de": "German Summary" 
  },
  "example2.com": {
   "summary_en": "English Summary",
   "summary_de": "German Summary"
  }
}"""

# # convert the first 5 entries from df_dict into a string
# df_dict_5 = {k: v for k, v in list(df_dict.items())[:500]}
# df_dict_5_str = json.dumps(df_dict_5)
# df_dict_str = json.dumps(df_dict)

import json
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_legacy_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=api_legacy_key)

# Function to process a batch of data
def process_batch(batch):
    try:
        # Convert batch to JSON string
        batch_str = json.dumps(batch)

        # Make API request
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": summary_prompt},
                {"role": "user", "content": sum_input_example},
                {"role": "assistant", "content": sum_output_example},
                {"role": "user", "content": batch_str}
            ],
            response_format={"type": "json_object"},
            temperature=0.2,
        )

        # Return the entire message content
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing batch: {e}")
        return ""

# Function to divide dictionary into batches
def divide_into_batches(data_dict, batch_size):
    items = list(data_dict.items())
    for i in range(0, len(items), batch_size):
        yield dict(items[i:i + batch_size])

# Main processing
batch_size = 100
df_dict_batches = divide_into_batches(df_dict, batch_size)

# Initialize a text string to store results
results_text = ""

# Initialize a counter for batch number
batch_number = 1
# Process each batch and append results to the text string
for batch in df_dict_batches:
    print(f"Processing batch {batch_number}")
    batch_result = process_batch(batch)
    results_text += batch_result + "\n"
    batch_number += 1


# Save the results to a text file
with open("site_summary_results_round2.txt", "w") as file:
    file.write(results_text)

In [ ]:
import json

def read_json_objects(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    json_objects = []
    current_object = []
    open_braces = 0
    
    for line in lines:
        stripped_line = line.strip()
        current_object.append(line)
        open_braces += stripped_line.count('{')
        open_braces -= stripped_line.count('}')
        
        if open_braces == 0 and current_object:
            json_str = ''.join(current_object).strip()
            if json_str.endswith(','):
                json_str = json_str[:-1]
            try:
                json_obj = json.loads(json_str)
                json_objects.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON object: {e}")
                print(json_str)
            current_object = []
    
    print(f"Total JSON objects read: {len(json_objects)}")  # Debugging statement
    return json_objects

def process_json_objects(json_objects):
    standardized_data = {}
    
    for obj in json_objects:
        if "summaries" in obj:
            for summary in obj["summaries"]:
                standardized_data[summary["domain"]] = {
                    "summary_en": summary["summary_en"],
                    "summary_de": summary["summary_de"]
                }
        else:
            for domain, summaries in obj.items():
                standardized_data[domain] = {
                    "summary_en": summaries["summary_en"],
                    "summary_de": summaries["summary_de"]
                }
    
    print(f"Total entries after processing: {len(standardized_data)}")  # Debugging statement
    return standardized_data

def main(file_path):
    json_objects = read_json_objects(file_path)
    standardized_data = process_json_objects(json_objects)
    
    with open('standardized_site_summaries_round2.json', 'w') as file:
        json.dump(standardized_data, file, indent=2, ensure_ascii=False)
    
    print(f"Final total entries written: {len(standardized_data)}")  # Debugging statement

# Path to the faulty JSON file
file_path = 'site_summary_results_round2.txt'

# Run the main function
main(file_path)

In [ ]:
# remove entries from standardized_site_summaries.json where summary_en is empty
with open('standardized_site_summaries.json', 'r') as file:
    data = json.load(file)

data_filtered = {k: v for k, v in data.items() if v["summary_en"] != ""} 

with open('standardized_site_summaries_filtered.json', 'w') as file:
    json.dump(data_filtered, file, indent=2, ensure_ascii=False)




In [ ]:
# find which entries don't have a summary
# load the standardized_site_summaries.json
with open('standardized_site_summaries.json', 'r') as file:
    standardized_data = json.load(file)

# check unique_domains and remove those items that have a summary in standardized_data
unique_domains_no_summary = [domain for domain in unique_domains if domain not in standardized_data.keys()]
print(f"Unique domains without summary: {len(unique_domains_no_summary)}")

# add to unique_domains_no_summary those domains from standardized_data that have an empty summary
unique_domains_no_summary += [domain for domain, summary in standardized_data.items() if summary["summary_en"] == ""]

# print length of unique_domains_no_summary
print(f"Unique domains without summary: {len(unique_domains_no_summary)}")

In [ ]:
import nest_asyncio
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Function to format URLs
def format_url(url):
    if not url.startswith(('http://', 'https://')):
        url = 'http://' + url
    return url

# Function to clean text
def clean_text(text):
    return ' '.join(text.split())

# Function to try different URL formats
async def try_different_formats(session, url, headers):
    formats = [
        url,
        url.replace('http://', 'https://'),
        'http://www.' + urlparse(url).netloc,
        'https://www.' + urlparse(url).netloc
    ]
    for fmt in formats:
        try:
            async with session.get(fmt, headers=headers, timeout=10) as response:
                if response.status == 200:
                    html = await response.text()
                    soup = BeautifulSoup(html, 'html.parser')
                    text = soup.get_text()
                    cleaned_text = clean_text(text[:600])  # Clean the text
                    return fmt, cleaned_text
        except Exception:
            time.sleep(1)  # Introduce a delay before retrying
            continue
    return url, "Connection failed: All formats tried"

# Function to fetch and parse a single URL
async def fetch(session, url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }
    url = format_url(url)  # Format the URL
    try:
        async with session.get(url, headers=headers, timeout=15) as response:
            if response.status == 200:
                html = await response.text()
                soup = BeautifulSoup(html, 'html.parser')
                text = soup.get_text()
                cleaned_text = clean_text(text[:600])  # Clean the text
                return url, cleaned_text  # Return the URL and the cleaned text
            else:
                return await try_different_formats(session, url, headers)
    except aiohttp.ClientError:
        return await try_different_formats(session, url, headers)
    except Exception as e:
        return url, f"Connection failed: {str(e)}"

# Function to handle fetching multiple URLs
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            task = asyncio.ensure_future(fetch(session, url))
            tasks.append(task)
            await asyncio.sleep(0.1)  # Introduce a small delay after each request
        responses = await asyncio.gather(*tasks)
        return responses

# List of 5000 URLs to visit
urls = unique_domains_no_summary

# Run the scraper
responses = asyncio.run(fetch_all(urls))

# Convert results to a DataFrame
df = pd.DataFrame(responses, columns=["Domain", "Text"])

# Evaluate the results
successful_connections = df[df["Text"].str.contains("Connection failed") == False].shape[0]
failed_connections = df[df["Text"].str.contains("Connection failed")].shape[0]

print(f"Successful connections: {successful_connections}")
print(f"Failed connections: {failed_connections}")

# Display the first few rows of the dataframe
print(df.head())

# Save the results to a CSV file
df.to_csv("scraped_data_round2.csv", index=False)

In [ ]:
import csv
import json

# Assuming response_summary_sample.choices[0].message.content is a JSON string
json_content = response_summary_sample.choices[0].message.content

# Parse the JSON string into a Python object (list of dictionaries)
data = json.loads(json_content)

# Store results in a CSV file
with open("summary_results.csv", "w", newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["domain", "summary_en", "summary_de"])
    writer.writeheader()
    for entry in data['summaries']:
        writer.writerow(entry)

In [ ]:
# 5.3 Automatic link categorization with GPT-4o 
## Comparison of two approaches: just the domains, and the domains with the first 500 characters of text.

## Preparation of the prompts

intro_prompt = """You are a categorizer of education web domains into predefined categories that only outputs in JSON. 
They are domains that are in the education ecosystem of Austria. The categories are linked to the roles they play in the innovation ecosystem.
The categories you can choose from are:"""

categories_prompt = """{
    "National Ministry and Regulatory Bodies": {
        "roles": ["Customers"],
        "description": "Government entities responsible for overseeing and regulating education at the national level."
    },
    "Local Authorities": {
        "roles": ["Customers", "Suppliers"],
        "description": "Regional or local government bodies that administer and support schools in their area."
    },
    "Parent / Alumni Associations": {
        "roles": ["Customers"],
        "description": "Groups formed by parents and alumni to support school activities and represent community engagement."
    },
    "Learning Management Systems and Administrative Tools": {
        "roles": ["Suppliers"],
        "description": "Software solutions that help schools manage and deliver educational content and administrative tasks."
    },
    "Learning and Teaching Material Providers": {
        "roles": ["Suppliers"],
        "description": "Organizations that supply educational materials, including textbooks, digital content, and teaching aids."
    },
    "Learning Experience Provider": {
        "roles": ["Complementors"],
        "description": "Entities offering supplementary educational experiences to students, such as workshops, courses, or extracurricular programs. Excludes businesses not related to schools."
    },
    "(Social) Media Organizations and Platforms": {
        "roles": ["Complementors"],
        "description": "Platforms and organizations that facilitate communication and content sharing around educational communities."
    },
    "Web Development Companies, Photography Services": {
        "roles": ["Suppliers"],
        "description": "Businesses providing digital and creative services like website development and photography. Excludes specialised educational services and platforms."
    },
    "Psychological Counselling": {
        "roles": ["Suppliers"],
        "description": "(Psychologists, systemic therapists, psychotherapists and organizations offering mental health services and support to students and school staff."
    },
    "Business Partners and Representation": {
        "roles": ["Complementors"],
        "description": "Companies and organizations not directly related to schools, excluding psychologists and psychotherapists."
    },
    "Inter-School Organizations and International Networks": {
        "roles": ["Complementors"],
        "description": "Networks that connect schools for the purpose of collaboration, resource sharing, and international exchange."
    },
    "Other Schools": {
        "roles": ["Schools"],
        "description": "Other official school institutions that might offer options for students to continue studying there."
    },
    "Research, Academia, and Teacher Training": {
        "roles": ["Suppliers"],
        "description": "Institutions / universities involved in educational research, higher education, and the professional development of teachers."
    }
}"""

intermediate_prompt_without_unsure = """Exclusively choose from the above categories.
To give orientation for your categorising: Customers pay for or request the educational services, Suppliers provide relevant inputs to the schools, and Complementors provide offers directly to the school and students.
Do not choose Customers, Suppliers, Complementors or Other Roles as categories! They are there to help you select the right categories (key) from the list above.
Do not create new categories."""

intermediate_prompt_with_unsure = """Exclusively choose from the above categories. Don't guess, just respond with 'Unsure' if you do not have the information in your training data.
To give orientation for your categorising: Customers pay for or request the educational services, Suppliers provide relevant inputs to the schools, and Complementors provide offers directly to the school and students.
Do not choose Customers, Suppliers, Complementors or Other Roles as categories! They are there to help you select the right categories (key) from the list above.
Do not create new categories."""

prompt_with_unsure = intro_prompt + categories_prompt + intermediate_prompt_with_unsure
prompt_without_unsure = intro_prompt + categories_prompt + intermediate_prompt_without_unsure

example_input_plain = """
klio.webuntis.com
bundeskanzleramt.gv.at
bfdi.bund.de
newsletter.wko.at
technikum-wien.at
cba.media
facebook.com
frauenberatung.at
humwien.at
www1.schabi.ch
bildung.bmbwf.gv.at
ecoschools.global
demokratiewebstatt.at
de.wikipedia.org
wiener-hak.at
canva.com
search.follettsoftware.com
stem.org.uk
kastner.at
marianisten.at
ms.bcfries.at
bildung-wien.gv.at
bmbwf.gv.at
waisenversorgungsverein.org
vip-uebungsfirma.jimdo.com
jmtt.phwien.net
voxmi.at
card2brain.ch
rabenhoftheater.com
oe1.orf.at
bmb.gv.at
theaterimpark.at
akademie-biberkor.de
talentify.me
lasalle.org
bmbwf.gv.at
kurier.at
digitaleslernen.oead.at
webmail.g19.at
dsb.gv.at
matura.gv.at
internationalcampusvienna.at
devmultisite.superfesch.at
twyn.com
steinedererinnerung.net
facebook.com
polgargym-my.sharepoint.com
wohnpartner-wien.at
youtube.com
forms.office.com"""

example_output_without_unsure = """{ "klio.webuntis.com": "Learning Management Systems and Administrative Tools",
  "bundeskanzleramt.gv.at": "National Ministry and Regulatory Bodies",
  "bfdi.bund.de": "National Ministry and Regulatory Bodies",
  "newsletter.wko.at": "Business Partners and Representation",
  "technikum-wien.at": "Research, Academia, and Teacher Training",
  "cba.media": "(Social) Media Organizations and Platforms",
  "facebook.com": "(Social) Media Organizations and Platforms",
  "frauenberatung.at": "Psychological Counselling",
  "humwien.at": "Inter-School Organizations and International Networks",
  "www1.schabi.ch": "Learning and Teaching Material Providers",
  "bildung.bmbwf.gv.at": "National Ministry and Regulatory Bodies",
  "ecoschools.global": "Inter-School Organizations and International Networks",
  "demokratiewebstatt.at": "Learning Experience Provider",
  "de.wikipedia.org": "Learning Material Provider",
  "wiener-hak.at": "Inter-School Organizations and International Networks",
  "canva.com": "Web Development Companies, Photography Services",
  "search.follettsoftware.com": "Learning and Teaching Material Providers",
  "stem.org.uk": "Learning and Teaching Material Providers",
  "kastner.at": "Business Partners and Representation",
  "marianisten.at": "Other Schools",
  "ms.bcfries.at": "Other Schools",
  "bildung-wien.gv.at": "Local Authorities",
  "bmbwf.gv.at": "National Ministry and Regulatory Bodies",
  "waisenversorgungsverein.org": "Learning Experience Provider",
  "vip-uebungsfirma.jimdo.com": "Web Development Companies, Photography Services",
  "jmtt.phwien.net": "Research, Academia, and Teacher Training",
  "voxmi.at": "Learning and Teaching Material Providers",
  "card2brain.ch": "Learning and Teaching Material Providers",
  "rabenhoftheater.com": "Learning Experience Provider",
  "oe1.orf.at": "(Social) Media Organizations and Platforms",
  "bmb.gv.at": "National Ministry and Regulatory Bodies",
  "theaterimpark.at": "Learning Experience Provider",
  "akademie-biberkor.de": "Research, Academia, and Teacher Training",
  "talentify.me": "Learning Experience Provider",
  "lasalle.org": "Other Schools",
  "bmbwf.gv.at": "National Ministry and Regulatory Bodies",
  "kurier.at": "(Social) Media Organizations and Platforms",
  "digitaleslernen.oead.at": "Learning and Teaching Material Providers",
  "webmail.g19.at": "Learning Management Systems and Administrative Tools",
  "dsb.gv.at": "National Ministry and Regulatory Bodies",
  "matura.gv.at": "National Ministry and Regulatory Bodies",
  "internationalcampusvienna.at": "Inter-School Organizations and International Networks",
  "devmultisite.superfesch.at": "Web Development Companies, Photography Services",
  "twyn.com": "Learning Management Systems and Administrative Tools",
  "steinedererinnerung.net": "Learning Experience Provider",
  "facebook.com": "(Social) Media Organizations and Platforms",
  "polgargym-my.sharepoint.com": "Learning Management Systems and Administrative Tools",
  "wohnpartner-wien.at": "Local Authorities",
  "youtube.com": "(Social) Media Organizations and Platforms",
  "forms.office.com": "Learning Management Systems and Administrative Tools"
}"""
example_output_with_unsure = """{ "klio.webuntis.com": "Learning Management Systems and Administrative Tools",
  "bundeskanzleramt.gv.at": "National Ministry and Regulatory Bodies",
  "bfdi.bund.de": "National Ministry and Regulatory Bodies",
  "newsletter.wko.at": "Business Partners and Representation",
  "technikum-wien.at": "Research, Academia, and Teacher Training",
  "cba.media": "(Social) Media Organizations and Platforms",
  "facebook.com": "(Social) Media Organizations and Platforms",
  "frauenberatung.at": "Psychological Counselling",
  "humwien.at": "Inter-School Organizations and International Networks",
  "www1.schabi.ch": "Learning and Teaching Material Providers",
  "bildung.bmbwf.gv.at": "National Ministry and Regulatory Bodies",
  "ecoschools.global": "Inter-School Organizations and International Networks",
  "demokratiewebstatt.at": "Learning Experience Provider",
  "de.wikipedia.org": "Unsure",
  "wiener-hak.at": "Inter-School Organizations and International Networks",
  "canva.com": "Web Development Companies, Photography Services",
  "search.follettsoftware.com": "Learning and Teaching Material Providers",
  "stem.org.uk": "Learning and Teaching Material Providers",
  "kastner.at": "Business Partners and Representation",
  "marianisten.at": "Other Schools",
  "ms.bcfries.at": "Other Schools",
  "bildung-wien.gv.at": "Local Authorities",
  "bmbwf.gv.at": "National Ministry and Regulatory Bodies",
  "waisenversorgungsverein.org": "Unsure",
  "vip-uebungsfirma.jimdo.com": "Web Development Companies, Photography Services",
  "jmtt.phwien.net": "Research, Academia, and Teacher Training",
  "voxmi.at": "Learning and Teaching Material Providers",
  "card2brain.ch": "Learning and Teaching Material Providers",
  "rabenhoftheater.com": "Learning Experience Provider",
  "oe1.orf.at": "(Social) Media Organizations and Platforms",
  "bmb.gv.at": "National Ministry and Regulatory Bodies",
  "theaterimpark.at": "Learning Experience Provider",
  "akademie-biberkor.de": "Research, Academia, and Teacher Training",
  "talentify.me": "Learning Experience Provider",
  "lasalle.org": "Other Schools",
  "bmbwf.gv.at": "National Ministry and Regulatory Bodies",
  "kurier.at": "(Social) Media Organizations and Platforms",
  "digitaleslernen.oead.at": "Learning and Teaching Material Providers",
  "webmail.g19.at": "Learning Management Systems and Administrative Tools",
  "dsb.gv.at": "National Ministry and Regulatory Bodies",
  "matura.gv.at": "National Ministry and Regulatory Bodies",
  "internationalcampusvienna.at": "Inter-School Organizations and International Networks",
  "devmultisite.superfesch.at": "Web Development Companies, Photography Services",
  "twyn.com": "Learning Management Systems and Administrative Tools",
  "steinedererinnerung.net": "Learning Experience Provider",
  "facebook.com": "(Social) Media Organizations and Platforms",
  "polgargym-my.sharepoint.com": "Learning Management Systems and Administrative Tools",
  "wohnpartner-wien.at": "Local Authorities",
  "youtube.com": "(Social) Media Organizations and Platforms",
  "forms.office.com": "Learning Management Systems and Administrative Tools"
}"""

example_input_webtext = """
{'stkarl.nikolausstiftung.at': 'St. Nikolausstiftung - Kindergarten St. Karl: Unser Haus Link zu: Kindergarten St. Karl: Unser HausUnser Haus Link zu: Kindergarten St. Karl: PädagogikPädagogik Link zu: Kindergarten St. Karl: FotosFotos Folge einem manuell hinzugefügten LinkKontakt ZurückWeiter123Herzlich willkommen im Pfarrkindergarten St. Karl Unsere Gruppe Team Informationen zur Anmeldung Informationen zu den Kosten Öffnungszeiten Wir freuen uns darauf, Sie und Ihr Kind kennenzulernen! Unser eingruppiger Kindergarten bietet eine sehr familiäre und persönliche Atmosphäre und ein Klima, das Freundlichkeit, Geborgenheit und Vertrautheit aufkommen lässt. In unserer Kindergartengruppe werden drei- bis sechsjährige Kinder betreut. Der besonders große Gruppenraum und der Turns'},
{'medienmonster.info': 'Willkommen bei den MedienMonstern Projekte Leistungen Projekte für Kinder & Jugendliche Workshops für Fachkräfte Elternabende / Vorträge Materialien / Konzepte Schulentwicklung Angebotsbroschüre Partner Kooperationspartner Bildungsinstitutionen Förderer Über uns Team Machen Sie mit! Kontakt Kreative Medienbildung Medienprojekte, Fortbildungen & Materialien Zu den Angeboten Willkommen bei den MedienMonstern Die MedienMonster sind ein gemeinnütziger Verein, der mit starken Projekten den kreativen und sozialen Umgang mit neuen Medien bei Kindern und Jugendlichen fördert. 150 Bildungs-einrichtungen 9000 Kinder &Jugendliche 3500 PädagogischeFachkräfte 1250 Eltern Projekte Mit Games Geschichten erzählen An der Junior Uni Essen haben wir mit Jugen'},
{'cbg-mittelhessen.de': '- CBG Mittelhessen CBG Mittelhessen Christliches Bildungszentrum für Gesundheitsberufe Menu + Das CBG + Die Schule Unser Team Jobs Die Ausbildung + Allgemeines Inhalte & Ziele Ausbildungsorte Was danach? Kontakt + Impressum Datenschutzerklärung Herzlich Willkommen auf den Seiten des Christlichen Bildungszentrums für Gesundheitsberufe (CBG Mittelhessen). Wir bilden Pflegefachfrauen und Pflegefachmänner (und divers) aus. Theorie und Praxis werden dabei optimal verknüpft: Während der dreijährigen Ausbildung findet der Theorieteil, im Wechsel mit den praktischen Einsätzen, in jeweils mehrwöchigen Unterrichtsblöcken am CBG statt. Im praktischen Teil durchlaufen die Auszubildenden die verschiedenen Stationen und Bereiche der jeweiligen Trägerkran'},
{'ist.or.at': 'Institut für Systemische Therapie – Psychotherapie Beratung Supervision Coaching Skip to content KontaktLinksImpressumAGBsDatenschutzKlientInneninformation Institut für Systemische Therapie HomeVeranstaltungen AnmeldungHoffen, stärken, verändernCurriculum Systemische GruppentherapieSystemische Gruppentherapie. Einführung und RefresherAufmerksamkeitsfokussierung erzeugt UnterschiedeUnterschiedliches Wahrnehmen, Fühlen und Denken.Ein simultanes GruppentherapiekonzeptMethoden, die einen Unterschied machenWie viele Geschlechter kennt die Psychotherapie?Fürchte Dich nicht – so sehr!Trauma und Wirklichkeiten36 Stunden live dabei! Sommer-Intensiv-Training (SIT) 2024Unterschiede als RessourceLösungsorientierte Beratung nach systemischen GrundlagenÜ'}"""

example_output_webtext = """
{'stkarl.nikolausstiftung.at': 'Other Schools',
'medienmonster.info': 'Education Experience Provider',
'cbg-mittelhessen.de': 'Other Schools'
'ist.or.at': 'Psychological Counselling'}"""

# Testing original web test vs. a summary

example_input_summary = """{'stkarl.nikolausstiftung.at': 'St. Nikolausstiftung operates a kindergarten focusing on pedagogical approaches and community involvement.'},
{'medienmonster.info': 'MedienMonster offers media education projects and workshops for children and youth, focusing on creative media skills.'},
{'cbg-mittelhessen.de': 'CBG Mittelhessen is a Christian educational center for health professions, offering training programs.'},
{'ist.or.at': 'The Institute for Systemic Therapy offers psychotherapy, counseling, and training programs focusing on systemic approaches.'}"""

example_output_summary = """
{'stkarl.nikolausstiftung.at': 'Category',
'medienmonster.info': 'Category',
'cbg-mittelhessen.de': 'Category'
'ist.or.at': 'Category'}"""

## Testing the prompts

testset_input = """appserei.com
915053-redaktion.schulen.wien.gv.at
paysafecard.com
oeamtc.at
baumgarten.nikolausstiftung.at
ritakern.at
music.amazon.de
quiz.westermann.de
lerntrick.com
krcma.at
jochenfallmann.at
ifa.or.at
meduniwien.ac.at
ante-portas.at
ista.ac.at
praxis-widauer.at
lasalle-assedil.org
bildungszentrum-kenyongasse.at
diepresse.com
das.fotovonzinner.com
bfi.at
kija-wien.at
kohaut.jimdofree.com
efa.vor.at
moodle.htlw10.at
signon.springer.com
moodle.lehrerweb.at
bmj.gv.at
lernmax.at
merlin-technology.com
stories.audible.com
ita.or.at
homepage.bildungsserver.com
businessatschool.de
iwik.at
vwgh.gv.at
praxis-psychotherapie-mediation.at
waldorfschule-poetzleinsdorf.at
kkunert.com
orthografietrainer.net
janegoodall.at
chorusviennensis.at
ulrikereisner.com
kunterbuntewelt.at
regionews.at
dasjetzt.at
juliascherbaum.at
gesundheitskasse.at
help.instagram.com
openthesaurus.de
popperabsolventen.com
lehrstelleninfo.at
digi4school.com
download.moodle.org
weckenmann.at
hvimeo.com
religion.orf.at
wwwpolgargym.at
gafa-absolventen.at
seeseiten.buchkatalog.at
goldenetramway.at
oefeb2024.phwien.ac.at
914121.schulen.wien.gv.at
oebv.at
umweltbildung.at
esra.at
apcoa.at
news.microsoft.com
app.classninjas.com
steiner-hitech.at
datenschutz-grundverordnung.eu
wienzufuss.at
wienersommerdeutschkurse.at
conservatoriocimarosa.org
weltweitunterrichten.at
iro.ogu.edu.tr
onb.ac.at
geschichtsdidaktik.eu
areeka.net
bohrnpatricia.net
co-paartherapie.at
stkarl.nikolausstiftung.at
medienmonster.info
cbg-mittelhessen.de
ist.or.at
wegweiser-verlag.at
langegger-dick.at
humane-balance.at
fachhochschulen.ac.at
unicef.at
christophschwarz.net
hmc.org.uk
psychotherapie-stpoelten.at
privatakademie.de
gabriella-walisch.at
eeducation.at
elternvereinwittelsbach.org
tge-online.de
fotoalbum.himmelhof-wien.at
ifl.at"""

testset_output = """{
    "appserei.com": "Web Development Companies, Photography Services",
    "915053-redaktion.schulen.wien.gv.at": "Local Authorities",
    "paysafecard.com": "Business Partners and Representation",
    "oeamtc.at": "Business Partners and Representation",
    "baumgarten.nikolausstiftung.at": "Other Schools",
    "ritakern.at": "Psychological Counselling",
    "music.amazon.de": "(Social) Media Organizations and Platforms",
    "quiz.westermann.de": "Learning and Teaching Material Providers",
    "lerntrick.com": "Learning and Teaching Material Providers",
    "krcma.at": "Psychological Counselling",
    "jochenfallmann.at": "Psychological Counselling",
    "ifa.or.at": "Inter-School Organizations and International Networks",
    "meduniwien.ac.at": "Research, Academia, and Teacher Training",
    "ante-portas.at": "Business Partners and Representation",
    "ista.ac.at": "Research, Academia, and Teacher Training",
    "praxis-widauer.at": "Psychological Counselling",
    "lasalle-assedil.org": "Other Schools",
    "bildungszentrum-kenyongasse.at": "Other Schools",
    "diepresse.com": "(Social) Media Organizations and Platforms",
    "das.fotovonzinner.com": "Web Development Companies, Photography Services",
    "bfi.at": "Research, Academia, and Teacher Training",
    "kija-wien.at": "Local Authorities",
    "kohaut.jimdofree.com": "Learning and Teaching Material Providers",
    "efa.vor.at": "Local Authorities",
    "moodle.htlw10.at": "Learning Management Systems and Administrative Tools",
    "signon.springer.com": "Learning and Teaching Material Providers",
    "moodle.lehrerweb.at": "Learning Management Systems and Administrative Tools",
    "bmj.gv.at": "National Ministry and Regulatory Bodies",
    "lernmax.at": "Learning and Teaching Material Providers",
    "merlin-technology.com": "Business Partners and Representation",
    "stories.audible.com": "(Social) Media Organizations and Platforms",
    "ita.or.at": "Research, Academia, and Teacher Training",
    "homepage.bildungsserver.com": "Learning and Teaching Material Providers",
    "businessatschool.de": "Learning Experience Provider",
    "iwik.at": "Psychological Counselling",
    "vwgh.gv.at": "National Ministry and Regulatory Bodies",
    "praxis-psychotherapie-mediation.at": "Psychological Counselling",
    "waldorfschule-poetzleinsdorf.at": "Other Schools",
    "kkunert.com": "Psychological Counselling",
    "orthografietrainer.net": "Learning and Teaching Material Providers",
    "janegoodall.at": "Learning Experience Provider",
    "chorusviennensis.at": "Learning Experience Provider",
    "ulrikereisner.com": "Psychological Counselling",
    "kunterbuntewelt.at": "Psychological Counselling",
    "regionews.at": "(Social) Media Organizations and Platforms",
    "dasjetzt.at": "Business Partners and Representation",
    "juliascherbaum.at": "Psychological Counselling",
    "gesundheitskasse.at": "Business Partners and Representation",
    "help.instagram.com": "(Social) Media Organizations and Platforms",
    "openthesaurus.de": "Learning and Teaching Material Providers",
    "popperabsolventen.com": "Parent / Alumni Associations",
    "lehrstelleninfo.at": "Learning and Teaching Material Providers",
    "digi4school.com": "Learning and Teaching Material Providers",
    "download.moodle.org": "Learning Management Systems and Administrative Tools",
    "weckenmann.at": "Psychological Counselling",
    "hvimeo.com": "(Social) Media Organizations and Platforms",
    "religion.orf.at": "(Social) Media Organizations and Platforms",
    "wwwpolgargym.at": "Other Schools",
    "gafa-absolventen.at": "Parent / Alumni Associations",
    "seeseiten.buchkatalog.at": "Learning and Teaching Material Providers",
    "goldenetramway.at": "Learning Experience Provider",
    "oefeb2024.phwien.ac.at": "Research, Academia, and Teacher Training",
    "914121.schulen.wien.gv.at": "Local Authorities",
    "oebv.at": "Learning and Teaching Material Providers",
    "umweltbildung.at": "Learning and Teaching Material Providers",
    "esra.at": "Psychological Counselling",
    "apcoa.at": "Business Partners and Representation",
    "news.microsoft.com": "(Social) Media Organizations and Platforms",
    "app.classninjas.com": "Learning Management Systems and Administrative Tools",
    "steiner-hitech.at": "Business Partners and Representation",
    "datenschutz-grundverordnung.eu": "National Ministry and Regulatory Bodies",
    "wienzufuss.at": "Learning Experience Provider",
    "wienersommerdeutschkurse.at": "Learning Experience Provider",
    "conservatoriocimarosa.org": "Other Schools",
    "weltweitunterrichten.at": "Inter-School Organizations and International Networks",
    "iro.ogu.edu.tr": "Research, Academia, and Teacher Training",
    "onb.ac.at": "Research, Academia, and Teacher Training",
    "geschichtsdidaktik.eu": "Research, Academia, and Teacher Training",
    "areeka.net": "Learning and Teaching Material Providers",
    "bohrnpatricia.net": "Psychological Counselling",
    "co-paartherapie.at": "Psychological Counselling",
    "stkarl.nikolausstiftung.at": "Other Schools",
    "medienmonster.info": "Learning Experience Provider",
    "cbg-mittelhessen.de": "Other Schools",
    "ist.or.at": "Psychological Counselling",
    "wegweiser-verlag.at": "Learning and Teaching Material Providers",
    "langegger-dick.at": "Psychological Counselling",
    "humane-balance.at": "Psychological Counselling",
    "fachhochschulen.ac.at": "Research, Academia, and Teacher Training",
    "unicef.at": "Learning Experience Provider",
    "christophschwarz.net": "Web Development Companies, Photography Services",
    "hmc.org.uk": "Inter-School Organizations and International Networks",
    "psychotherapie-stpoelten.at": "Psychological Counselling",
    "privatakademie.de": "Other Schools",
    "gabriella-walisch.at": "Psychological Counselling",
    "eeducation.at": "Learning and Teaching Material Providers",
    "elternvereinwittelsbach.org": "Parent / Alumni Associations",
    "tge-online.de": "Learning and Teaching Material Providers",
    "fotoalbum.himmelhof-wien.at": "Web Development Companies, Photography Services",
    "ifl.at": "Other Schools"
}"""

In [ ]:
# 5.3.1 Categorization

# open standardized_site_summaries.json
import json

# Open and read the JSON file
with open('standardized_site_summaries.json', 'r') as file:
    standardized_data = json.load(file)

input_object = []
for line in testset_input.split("\n"):
    domain = line.strip()
    if line in standardized_data.keys():
        input_object.append({domain: standardized_data[domain]["summary_en"]})
    else:
        input_object.append({domain: ""})
        

response_4o = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": prompt_without_unsure},
    {"role": "user", "content": example_input_summary},
    {"role": "assistant", "content": example_output_summary},
    {"role": "user", "content": testset_input}
  ],
    response_format={
    "type": "json_object",  # Enforces JSON format
  }, 
  temperature=0.0,
)

print(response_4o.choices[0].message.content)

In [ ]:
import json

# Ensure response_4o.choices[0].message.content is a valid JSON string
response_content = response_4o.choices[0].message.content

try:
    # Parse the JSON string into a dictionary
    response_json = json.loads(response_content)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    response_json = {}

# testset_output is a JSON string, so we need to parse it into a dictionary
try:
    testset_output_json = json.loads(testset_output)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    testset_output_json = {}

# Compare to testset_output_json and count correct answers
correct = 0
for domain, category in response_json.items():
    if domain in testset_output_json:
        if category == testset_output_json[domain]:
            correct += 1

print(f"Correctly categorized domains: {correct} out of {len(testset_output_json)}")

# Print incorrect entries
for domain, category in response_json.items():
    if domain in testset_output_json:
        if category != testset_output_json[domain]:
            print(f"Domain: {domain}, Predicted: {category}, Correct: {testset_output_json[domain]}")

In [ ]:
# preparation of all domains dictionary with domains and summaries where available
domains_with_summaries = {}
with open('standardized_site_summaries.json', 'r') as file:
    standardized_data = json.load(file)
# get links from unique domains
# get summaries from standardized_site_summaries.json where available
# for link in unique_domains:
#     if link in standardized_data.keys():
#         domains_with_summaries[link] = standardized_data[link]["summary_en"]
#     else:
#         domains_with_summaries[link] = ""


# create dictionary with only domains that have summaries:
for link in standardized_data.keys():
    domains_with_summaries[link] = standardized_data[link]["summary_en"]

# #tests
print(len(domains_with_summaries))
# # print 5 entries of domains_with_summaries
for i, (domain, summary) in enumerate(domains_with_summaries.items()):
    print(f"{domain}: {summary}")
    if i == 4:
        break


# for a test run, we will use the first 100 domains
#domains_with_summaries_sample = dict(list(domains_with_summaries.items())[:40])


In [ ]:
import json

# Categorize those sites that don't have a category yet
domains_without_category = {}

# Open site_summaries_and_categories.json
with open('site_summaries_and_categories.json', 'r') as file:
    site_summaries_and_categories = json.load(file)

for domain, details in site_summaries_and_categories.items():
    # Check if 'category' key exists and if there's no category yet, add domain and summary_en to domains_without_category
    if 'category' not in details:
        domains_without_category[domain] = details['summary_en']

# Print the domains without category
print(domains_without_category)

In [ ]:
# categorization in batches of 100
import json
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_legacy_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=api_legacy_key)

# Function to process a batch of data
def process_batch(batch):
    try:
        # Convert batch to JSON string
        batch_str = json.dumps(batch)

        # Make API request
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt_without_unsure},
                {"role": "user", "content": example_input_summary},
                {"role": "assistant", "content": example_output_summary},
                {"role": "user", "content": batch_str}
            ],
            response_format={"type": "json_object"},
            temperature=0.1,
        )

        # Return the entire message content
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error processing batch: {e}")
        return ""

# Function to divide dictionary into batches
def divide_into_batches(data_dict, batch_size):
    items = list(data_dict.items())
    for i in range(0, len(items), batch_size):
        yield dict(items[i:i + batch_size])

# Main processing
batch_size = 200
df_dict_batches = divide_into_batches(domains_without_category, batch_size)

# Initialize a text string to store results
results_text = ""

# Initialize a counter for batch number
batch_number = 1
# Process each batch and append results to the text string
for batch in df_dict_batches:
    print(f"Processing batch {batch_number}")
    batch_result = process_batch(batch)
    results_text += batch_result + "\n"
    batch_number += 1


# Save the results to a text file
with open("site_categorization_results_without_categories.txt", "w") as file:
    file.write(results_text)


In [ ]:
import json

# Read from site_categorization_results.txt
with open('site_categorization_results_without_categories.txt', 'r') as file:
    categorization_text = file.read()

# Add the results to a dictionary
results_dict = json.loads(categorization_text)

# Load standardized_site_summaries.json
with open('site_summaries_and_categories.json', 'r') as file:
    sites_and_summaries = json.load(file)

#print(sites_and_summaries["uszeged.hu"])

# Extend the standardized_site_summaries.json with the categories from the categorization
for domain, category in results_dict.items():
    if domain in sites_and_summaries:
        # Only update the category if it doesn't already exist or is empty
        if not sites_and_summaries[domain].get("category"):
            sites_and_summaries[domain]["category"] = category
    else:
        sites_and_summaries[domain] = {"summary_en": "", "category": category}

# Save the extended dictionary to a new file, while keeping special characters readable instead of escaped
with open("site_summaries_and_categories2.json", "w") as file:
    json.dump(sites_and_summaries, file, ensure_ascii=False, indent=4)

# 6. Visualisation of the results in graph form, based on different filters 



In [ ]:
# sample data

# Load the site summaries and categories
vis_sum_and_cat = {"systempraxis.at": {
        "summary_en": "SYSTEMPRAXIS.AT offers systemic psychotherapy and coaching services aimed at helping individuals navigate difficult times and find direction in life. They emphasize the importance of taking the first step towards personal growth and change.",
        "summary_de": "SYSTEMPRAXIS.AT bietet systemische Psychotherapie und Coaching-Dienste an, die darauf abzielen, Menschen in schwierigen Zeiten zu helfen und Orientierung im Leben zu finden. Sie betonen die Bedeutung des ersten Schrittes in Richtung persönlicher Entwicklung und Veränderung.",
        "category": "Psychological Counselling"
    },
    "abendgym-klagenfurt.at": {
        "summary_en": "Abendgymnasium Klagenfurt is a public evening school offering free education and resources for students pursuing their Matura. They provide various study programs and support services for adult learners.",
        "summary_de": "Das Abendgymnasium Klagenfurt ist eine öffentliche Abendschule, die kostenlose Bildung und Ressourcen für Schüler anbietet, die ihre Matura anstreben. Sie bieten verschiedene Studienprogramme und Unterstützungsdienste für Erwachsene an.",
        "category": "Other Schools"
    },
    "cedefop.europa.eu": {
        "summary_en": "CEDEFOP is the European Centre for the Development of Vocational Training, focusing on skills and labor market policies. They provide resources and support for vocational education and training for youth and adults.",
        "summary_de": "CEDEFOP ist das Europäische Zentrum für die Entwicklung der Berufsbildung, das sich auf Fähigkeiten und Arbeitsmarktpolitik konzentriert. Sie bieten Ressourcen und Unterstützung für die berufliche Bildung und Ausbildung von Jugendlichen und Erwachsenen.",
        "category": "Research, Academia, and Teacher Training"
    },
    "grundschulkoenig.de": {
        "summary_en": "Grundschulkönig provides worksheets and teaching materials for elementary schools, offering a variety of free exercises in subjects like German, math, and general knowledge.",
        "summary_de": "Grundschulkönig bietet Arbeitsblätter und Unterrichtsmaterialien für Grundschulen an und stellt eine Vielzahl von kostenlosen Übungen in Fächern wie Deutsch, Mathe und Sachkunde zur Verfügung.",
        "category": "Learning and Teaching Material Providers"
    }}


vis_sample_school_list = [{
        "SKZ": "910021",
        "base_url": "https://alxinger.schule.wien.at",
        "internal_links": ["https://alxinger.schule.wien.at",
            "https://alxinger.schule.wien.at/something",
            "https://alxinger.schule.wien.at/somethingelse",
            "https://alxinger.schule.wien.at/somethingdifferent",],
        "visited_internal_links": [
            "https://alxinger.schule.wien.at/something",
            "https://alxinger.schule.wien.at/somethingelse"
        ],
        "attempted_visits_links": [
            "https://alxinger.schule.wien.at",
            "https://alxinger.schule.wien.at/something",
            "https://alxinger.schule.wien.at/somethingelse"
        ],
        "all_links": ["https://www.cedefop.europa.eu/en/about-cedefop/recruitment/traineeships",
                      "https://www.grundschulkoenig.de/",
                      "https://abendgym-klagenfurt.at",
                      "http://www.systempraxis.at",]
        }]

# VS Alxingergasse 82,1,Volksschule (öffentlich),http://alxinger.schule.wien.at/,"10., Alxingergasse 82",910021,Volksschule,öffentl.,https://alxinger.schule.wien.at/,False

In [ ]:
# FILTERING
import pandas as pd
comparison_df = pd.read_csv('comparison_df_2024-08-12.csv')

filtered_df = comparison_df[comparison_df["school_type_txt"] == "Allgemeinbildende höhere Schule (öffentlich)"]
# multiple schooltypes
# filtered_df = comparison_df[comparison_df["school_type_txt"].isin(["Bundesgymnasium", "Bundesrealgymnasium"])]


In [ ]:
#  Node-Edge Visualization Generator
from pyvis.network import Network
import networkx as nx


def vis_ne_loader(input_df: pd.DataFrame, output_name: str, language: str):
    summary_lang_string = "summary_" + language # en or de
    # load input data
    with open('scraped_links_2024-08-12_cleaned.json', 'r') as file:
        scraped_links = json.load(file)
    with open('site_summaries_and_categories.json', 'r') as file:
        site_summaries_and_categories = json.load(file)
    categories_dict = json.loads(categories_prompt)
    # Create a new PyVis network
    net = Network(height="100%", width="100%", bgcolor="#FFFFFF", font_color="black", notebook=True, cdn_resources='remote', select_menu=True)
    
    # Add nodes for each school
    for index, row in input_df.iterrows():
        # Find the matching dictionary
        try:
          matching_dict = next(d for d in scraped_links if d["SKZ"] == str(row["SKZ"]))
        except StopIteration:
          #print(f"SKZ {row['SKZ']} not found in scraped_links")
          continue
        
        if matching_dict["count_int_links"] == 0:
          continue

        # Print the base_url of the matching dictionary
        #print(matching_dict["base_url"])
        
        # Create the title
        title = (
            matching_dict["base_url"] + "\n" +
            row["school_type_txt"] + "\n" +
            row["address"] + "\n\nInternal Pages: " +
            str(matching_dict["count_int_links"]) + "\n\nExternal Domains:\n" +
            "\n".join(matching_dict["cleaned_ext_domains"])
        )
        
        # Add a node for the school
        net.add_node(label=row["name"], n_id=row["name"], title=title, group="Schools", shape="box")
        # Add a node for each external domain
        for domain in matching_dict["cleaned_ext_domains"]:
            # Get the category from the site summaries and categories
            try:
              existing_node = net.get_node(domain)
            except KeyError:
              if domain in site_summaries_and_categories:
                #print(domain)
                category = site_summaries_and_categories[domain]["category"]
                group = categories_dict[category]["roles"][0]
                summary = site_summaries_and_categories[domain][summary_lang_string]
                title = domain + " (" + category + ")\n\n" + summary + "\n\nPages linked to:\n"
                for link in matching_dict["cleaned_ext_links"]:
                    if link.startswith(domain):
                        title += link + "\n"
                net.add_node(n_id=domain, title=title, label=domain, group=group, shape="box")
                # add an edge between the school and the domain
                net.add_edge(row["name"], domain)

              else:
                  category = "Category not found"
                  title = "Couldn't reach the site \n\nPages linked to:\n"
            else: 
              net.add_edge(row["name"], domain)
              for link in matching_dict["cleaned_ext_links"]:
                if link.startswith(domain): 
                  existing_node["title"] += link + "\n"
            # Add the node

            # Add an edge between the school and the domain
            # net.add_edge(row["SKZ"], domain)



    # Set options for the visualization
    options = """
    var options = {
        "groups": {
            "Schools": {
                "color": {
                    "background": "#FA8029",
                    "border": "#F26D00"
                },
                "font": {
                    "color": "black"
                }
            },
            "Suppliers": {
                "color": {
                    "background": "#A8CA8F",
                    "border": "#9EC585"
                },
                "font": {
                    "color": "black"
                }
            },
            "Complementors": {
                "color": {
                    "background": "#FFBE00",
                    "border": "#FFD770"
                },
                "font": {
                    "color": "black"
                }
            },
            "Customers": {
                "color": {
                    "background": "#4892D2",
                    "border": "#4892D2"
                },
                "font": {
                    "color": "black"
                }
            }
        },
        "physics": {
            "enabled": true,
            "solver": "forceAtlas2Based",
            "forceAtlas2Based": {
                "gravity": -1000,
                "centralGravity": 0.025,
                "springLength": 150,
                "springStrength": 0.08,
                "damping": 0.9,
                "overlap": 0
            },
            "stabilization": {
                "enabled": true,
                "iterations": 750,
                "updateInterval": 25,
                "onlyDynamicEdges": false,
                "fit": true
            },
            "minVelocity": 0.1,
            "maxVelocity": 10,
            "timestep": 0.5,
            "adaptiveTimestep": true
        }
    }
    """
    #net.toggle_physics(False)
    net.set_options(options)
    net.prep_notebook()
    # net.show_buttons(filter_=['physics'])

    # Save the visualization to a file or display it in a Jupyter notebook
    net.show(output_name)



# vis_ne_loader(filtered_df, "6ahs_OEFF.html", "en")
# vis_ne_loader(comparison_df[comparison_df["school_type_txt"] == "Statutschule (privat)"], "6statutschulen_en.html", "en")
# vis_ne_loader(comparison_df[comparison_df["school_type_txt"] == "Volksschule (öffentlich)"], "6volksschulen_OEFF.html", "en")
# vis_ne_loader(comparison_df[comparison_df["school_type_txt"] == "Volksschule (privat)"], "6volksschulen_PRIV.html", "en")
# create url-friendly outputs
import re
import unicodedata
def slugify(value):
    # Normalize the string to remove accents
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    # Convert to lowercase
    value = value.lower()
    # Remove parentheses and other special characters
    value = re.sub(r'[^\w\s-]', '', value)
    # Replace spaces and underscores with hyphens
    value = re.sub(r'[-\s]+', '-', value)
    return value

print("\nNode - Edge school links in English and German:")

comparison_df_2024 = pd.read_csv('comparison_df_2024-08-12.csv')

# Create files for schooltypes with private and public aggregated in ENGLISH
for school_type in comparison_df_2024["school_type_mapping"].unique():
   output_filename = "6_" + slugify(school_type) + "_NE_en.html"
   print(school_type + " ENG")
   vis_ne_loader(comparison_df[comparison_df["school_type_mapping"] == school_type], output_filename, "en")

# Create files for schooltypes with separate schooltypes in ENGLISH
for school_type in comparison_df_2024["school_type_txt"].unique():
   output_filename = "6_" + slugify(school_type) + "_NE_en.html"
   print(school_type + " ENG")
   vis_ne_loader(comparison_df[comparison_df["school_type_txt"] == school_type], output_filename, "en")

   
# Create files for schooltypes with private and public aggregated in GERMAN
for school_type in comparison_df_2024["school_type_mapping"].unique():
   output_filename = "6_" + slugify(school_type) + "_NE_de.html"
   print(school_type + " DE")
   vis_ne_loader(comparison_df[comparison_df["school_type_mapping"] == school_type], output_filename, "de")

# Create files for schooltypes with separate schooltypes in GERMAN
for school_type in comparison_df_2024["school_type_txt"].unique():
   output_filename = "6_" + slugify(school_type) + "_NE_de.html"
   print(school_type + " DE")
   vis_ne_loader(comparison_df[comparison_df["school_type_txt"] == school_type], output_filename, "de")



# set output filename
# set language?
# get SKZ's from filtered_df
# get basic quantitative & qualitative stats for each SKZ 
# get links / domains for each SKZ from scraped_links_2024-08-12_cleaned.json
# get summary categories for each link domain from site_summaries_and_categories.json

# links_visited = vis_sample_school_list["SKZ"=="911021"]["attempted_visits_links"]
# print(links_visited)

In [ ]:
# print the unique school_type_mapping from comparison_df_2024-08-12.csv line by line
comparison_df_2024 = pd.read_csv('comparison_df_2024-08-12.csv')

for school_type in comparison_df_2024["school_type_mapping"].unique():
    print(school_type)

In [ ]:
#  Aggregate Visualization Generator

from pyvis.network import Network


def vis_agg_loader(input_df: pd.DataFrame, output_name: str, language: str):
    summary_lang_string = "summary_" + language # en or de
    # load input data
    with open('scraped_links_2024-08-12_cleaned.json', 'r') as file:
        scraped_links = json.load(file)
    with open('site_summaries_and_categories.json', 'r') as file:
        site_summaries_and_categories = json.load(file)
    categories_dict = json.loads(categories_prompt)
    # Create a new PyVis network
    net = Network(height="100%", width="100%", bgcolor="#FFFFFF", font_color="black", notebook=True, select_menu=True, cdn_resources='remote')
    
    # Create the category node for Schools
    # TODO: create the right shape. 
    net.add_node(label="Schools", n_id="Schools", title="Schools", group="Schools", shape="box", level=2, value=5, scaling={"label": {"enabled": True, "min": 5, "max": 50}})

    # Add nodes for each school
    for index, row in input_df.iterrows():
        # Find the matching dictionary
        try:
          matching_dict = next(d for d in scraped_links if d["SKZ"] == str(row["SKZ"]))
        except StopIteration:
          #print(f"SKZ {row['SKZ']} not found in scraped_links")
          continue
        
        if matching_dict["count_int_links"] == 0:
          continue

        # Print the base_url of the matching dictionary
        #print(matching_dict["base_url"])
        
        # Create the title
        title = (
            matching_dict["base_url"] + "\n" +
            row["school_type_txt"] + "\n" +
            row["address"] + "\n\nInternal Pages: " +
            str(matching_dict["count_int_links"]) + "\n\nExternal Domains:\n" +
            "\n".join(matching_dict["cleaned_ext_domains"])
        )
        
        # Add a node for the school
        net.add_node(label=row["name"], n_id=matching_dict["base_url"], title=title, group="Schools", shape="box", level=1)
        # Add an edge to Schools node
        net.add_edge("Schools", matching_dict["base_url"])
        # Increase size of Schools node
        #net.get_node("Schools")["value"] += 10
        # Add a node for each external domain
        for domain in matching_dict["cleaned_ext_domains"]:
            # Get the category from the site summaries and categories
            try:
              existing_node = net.get_node(domain)
            except KeyError:
              if domain in site_summaries_and_categories:
                #print(domain)
                category = site_summaries_and_categories[domain]["category"]
                group = categories_dict[category]["roles"][0]
                # Add category node if it doesn't exist
                try:
                  existing_node_cat = net.get_node(category)
                except KeyError:
                  net.add_node(label=category + " (" + group + ")", n_id=category, title="Description:" + categories_dict[category]["description"], group=group, shape="box", level=2, value=5, scaling={"label": {"enabled": True, "min": 5, "max": 50}})
                  net.add_edge("Schools", category, value=10)
                #else:
                  #existing_node_cat["value"] += 10
                summary = site_summaries_and_categories[domain][summary_lang_string]
                title = domain + " (" + category + ")\n\n" + summary + "\n\nPages linked to:\n"
                for link in matching_dict["cleaned_ext_links"]:
                    if link.startswith(domain):
                        title += link + "\n"
                net.add_node(n_id=domain, title=title, label=domain, group=group, shape="box", level=1)
                # add an edge between the school and the domain
                net.add_edge(category, domain)

              else:
                  category = "Category not found"
                  title = "Couldn't reach the site \n\nPages linked to:\n"
            else: 
            #   net.add_edge(row["SKZ"], domain)
              for link in matching_dict["cleaned_ext_links"]:
                if link.startswith(domain): 
                  existing_node["title"] += link + "\n"


    # Set options for the visualization
    options = """
    var options = {
        "groups": {
            "Schools": {
                "color": {
                    "background": "#FA8029",
                    "border": "#F26D00"
                },
                "font": {
                    "color": "black"
                }
            },
            "Suppliers": {
                "color": {
                    "background": "#A8CA8F",
                    "border": "#9EC585"
                },
                "font": {
                    "color": "black"
                }
            },
            "Complementors": {
                "color": {
                    "background": "#FFBE00",
                    "border": "#FFD770"
                },
                "font": {
                    "color": "black"
                }
            },
            "Customers": {
                "color": {
                    "background": "#4892D2",
                    "border": "#4892D2"
                },
                "font": {
                    "color": "black"
                }
            }
        }
    }
    """
    net.set_options(options)
    net.prep_notebook()
    # net.show_buttons(filter_=['physics'])
    #net.toggle_physics(False)

    # Save the visualization to a file or display it in a Jupyter notebook
    net.show(output_name)


# vis_agg_loader(filtered_df, "test.html", "en")

# create url-friendly outputs
import re
import unicodedata
def slugify(value):
    # Normalize the string to remove accents
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    # Convert to lowercase
    value = value.lower()
    # Remove parentheses and other special characters
    value = re.sub(r'[^\w\s-]', '', value)
    # Replace spaces and underscores with hyphens
    value = re.sub(r'[-\s]+', '-', value)
    return value

comparison_df_2024 = pd.read_csv('comparison_df_2024-08-12.csv')

print("\nAggregated school links in English and German:")

# Create files for schooltypes with private and public aggregated in ENGLISH
for school_type in comparison_df_2024["school_type_mapping"].unique():
   output_filename = "6_" + slugify(school_type) + "_AGG_en.html"
   print(school_type + " ENG")
   vis_agg_loader(comparison_df[comparison_df["school_type_mapping"] == school_type], output_filename, "en")

# Create files for schooltypes with separate schooltypes in ENGLISH
for school_type in comparison_df_2024["school_type_txt"].unique():
   output_filename = "6_" + slugify(school_type) + "_AGG_en.html"
   print(school_type + " ENG")
   vis_agg_loader(comparison_df[comparison_df["school_type_txt"] == school_type], output_filename, "en")

   
# Create files for schooltypes with private and public aggregated in GERMAN
for school_type in comparison_df_2024["school_type_mapping"].unique():
   output_filename = "6_" + slugify(school_type) + "_AGG_de.html"
   print(school_type + " DE")
   vis_agg_loader(comparison_df[comparison_df["school_type_mapping"] == school_type], output_filename, "de")

# Create files for schooltypes with separate schooltypes in GERMAN
for school_type in comparison_df_2024["school_type_txt"].unique():
   output_filename = "6_" + slugify(school_type) + "_AGG_de.html"
   print(school_type + " DE")
   vis_agg_loader(comparison_df[comparison_df["school_type_txt"] == school_type], output_filename, "de")




In [ ]:
# change the html file to work on Chrome too
import os

# Directory containing the HTML files
directory = "."

# The string to find and the replacement string
old_line = '<div class="card" style="width: 100%">'
new_line = '<div class="card" style="width: 100%; height: 100%">'

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.startswith("6_") and filename.endswith(".html"):
        filepath = os.path.join(directory, filename)
        
        # Read the contents of the file
        with open(filepath, "r") as file:
            content = file.read()
        
        # Replace the old line with the new line
        new_content = content.replace(old_line, new_line)
        
        # Write the modified content back to the file
        with open(filepath, "w") as file:
            file.write(new_content)

print("Replacement completed.")


In [ ]:
# sample visualisation to check if sizes work with the box
# Create a new PyVis network
net = Network(height="100%", width="100%", bgcolor="#FFFFFF", font_color="black", notebook=True, cdn_resources='remote', select_menu=True)
net.add_node(label="Schools", n_id="Schools", title="Schools", group="Schools", shape="box")
net.add_node(label="Category", n_id="Category", title="Category", group="Category", shape="box", value=10, scaling={"label": {"enabled": True, "min": 5, "max": 50}})
# net.add_node(label="Category", n_id="Category2", title="Category2", group="Category", shape="box", value=4, scaling={"min":0, "max":10, "label": {"enabled": True, "min": 5, "max": 50}, "maxVisible": 40})
net.add_node(label="Category", n_id="Category2", title="Category2", group="Category", shape="box", value=10, scaling={"min":0, "max":10, "label": {"enabled": True, "min": 5, "max": 50}})

net.add_node(label="School", n_id="School", title="School", group="School", shape="custom", )
net.add_edge("Schools", "Category")
net.show("test2.html")


# 7 Categorization Statistics
How many schools link to which categories
Exclude those schools with 0 internal links -> Start with links. 

In [ ]:
# percentages of schools having at least one type of a link
import pandas as pd
import json

# Load the schools dataset from the CSV file
comparison_df = pd.read_csv('comparison_df_2024-08-12.csv')

# Load the scraped links data from the JSON file
with open('scraped_links_2024-08-12_cleaned.json') as f:
    scraped_links = json.load(f)

# Convert scraped_links to DataFrame for easier processing
scraped_links_df = pd.DataFrame(scraped_links)

# Filter out schools with 'count_int_links' == 0
filtered_scraped_links_df = scraped_links_df[scraped_links_df['count_int_links'] > 0]

# Merge the filtered scraped links data with the schools dataset
merged_df = comparison_df.merge(filtered_scraped_links_df, on='SKZ', how='inner')

# Load the site summaries and categories
with open('site_summaries_and_categories.json') as f:
    site_summaries_and_categories = json.load(f)

# Map each domain to its category
domain_to_category = {domain: details['category'] for domain, details in site_summaries_and_categories.items()}

# Function to categorize external domains
def categorize_domains(domain_list, domain_to_category):
    categories = {category: 0 for category in set(domain_to_category.values())}
    for domain in domain_list:
        if domain in domain_to_category:
            category = domain_to_category[domain]
            categories[category] = 1
    return categories

# Apply the categorize_domains function to each school's external domains
merged_df['domain_categories'] = merged_df['cleaned_ext_domains'].apply(categorize_domains, args=(domain_to_category,))

# Convert the domain categories into separate columns
domain_categories_df = pd.json_normalize(merged_df['domain_categories'])

# Concatenate the original DataFrame with the domain categories DataFrame
result_df = pd.concat([merged_df[['school_type_txt']], domain_categories_df], axis=1)

# Calculate the percentage of each school type having links from each category
percentage_df = result_df.groupby('school_type_txt').mean() * 100

# Calculate the number of observations (n) for each school type
n_values = result_df.groupby('school_type_txt').size()

# Combine the number of observations with the percentage DataFrame
percentage_df['n'] = n_values

# Calculate the sum of n for the "Total" row and the mean of all other columns
total_row = percentage_df.mean(numeric_only=True).to_frame().T
total_row['n'] = n_values.sum()  # Use the sum for the 'n' column
total_row.index = ['Total']

# Concatenate the "Total" row with the percentage DataFrame
percentage_df = pd.concat([percentage_df, total_row])

# Generating LaTeX table code with overlapping diagonal headers and an additional "n" column
latex_code = "\\begin{adjustbox}{width=\\textwidth}\n\\begin{tabular}{l|c|" + "c|" * len(percentage_df.columns) + "}\n\\toprule\n"

# Create overlapping diagonal column headers using \makebox with invisible borders
latex_code += "School Type & n & "
for col in percentage_df.columns[:-1]:  # Exclude the "n" column from diagonal headers
    latex_code += f"\\makebox[2.2cm][l]{{\\rotatebox{{40}}{{{col}}}}} & "
latex_code = latex_code.rstrip("& ") + " \\\\\n\\midrule\n"

# Add data rows
for index, row in percentage_df.iterrows():
    latex_code += f"{index} & {int(row['n'])} & " + " & ".join(f"{val:.2f}\\%" for val in row[:-1]) + " \\\\\n"

# Add a final "Total" row with average values
total_row = percentage_df.mean().to_frame().T
total_row.index = ['Total']
latex_code += "\\midrule\n"
latex_code += f"Total & {int(total_row['n'].sum())} & " + " & ".join(f"{val:.2f}\\%" for val in total_row.iloc[0, :-1]) + " \\\\\n"

latex_code += "\\bottomrule\n\\end{tabular}\n\\end{adjustbox}"

# Display the generated LaTeX code
print(latex_code)



In [ ]:
# unique domains 
import pandas as pd
import json
from collections import defaultdict

# Load the data
comparison_df = pd.read_csv('comparison_df_2024-08-12.csv')
with open('scraped_links_2024-08-12_cleaned.json', 'r') as file:
    scraped_links = json.load(file)
with open('site_summaries_and_categories.json', 'r') as file:
    site_summaries_and_categories = json.load(file)

# Initialize a dictionary to store the links
category_links = defaultdict(lambda: defaultdict(list))

# Process each school in the comparison_df
for _, row in comparison_df.iterrows():
    school_category = row['school_type_mapping']
    skz = str(row['SKZ'])
    
    # Find the matching dictionary in scraped_links
    matching_dict = next((d for d in scraped_links if d["SKZ"] == skz), None)
    if not matching_dict:
        continue
    
    # Get the unique domains linked to
    unique_domains = set(matching_dict["cleaned_ext_domains"])
    
    # Collect the links in lists
    for domain in unique_domains:
        if domain in site_summaries_and_categories:
            category = site_summaries_and_categories[domain]["category"]
            category_links[school_category][category].append(domain)

# Convert lists to sets to ensure uniqueness and count the unique links
category_counts = defaultdict(lambda: defaultdict(int))
total_unique_domains = defaultdict(set)

for school_category, categories in category_links.items():
    for category, links in categories.items():
        unique_links = set(links)
        category_counts[school_category][category] = len(unique_links)
        total_unique_domains[category].update(unique_links)

# Create a DataFrame from the category counts
category_counts_df = pd.DataFrame(category_counts).fillna(0).astype(int)

# Transpose the DataFrame to switch rows and columns
category_counts_df = category_counts_df.T

# Calculate the total unique domains for each category
total_counts = {category: len(domains) for category, domains in total_unique_domains.items()}
total_counts_df = pd.DataFrame(total_counts, index=['Total (Unique)'])

# Concatenate the total counts row to the DataFrame
category_counts_df = pd.concat([total_counts_df, category_counts_df])

# Transpose the DataFrame to switch rows and columns
category_counts_df = category_counts_df.T


# Generate the LaTeX table without borders
latex_table = category_counts_df.to_latex(index=True, header=True, escape=False, column_format='c' * (len(category_counts_df.columns) + 1))

# Remove the \toprule, \midrule, and \bottomrule commands to eliminate borders
latex_table = latex_table.replace('\\toprule', '')
latex_table = latex_table.replace('\\midrule', '')
latex_table = latex_table.replace('\\bottomrule', '')
latex_table = latex_table.replace('\\hline', '')

# Remove commas from numbers
latex_table = latex_table.replace(',', '')

# Wrap the table with \resizebox to fit the page width
latex_table = f"\\resizebox{{\\textwidth}}{{!}}{{{latex_table}}}"

# Print the LaTeX table
print(latex_table)
